In [1]:
import pandas as pd
import numpy as np
import Memory as mm
import time
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp
import xgboost as xgb
import lightgbm as lgb
import math

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import sklearn.tree as tree
from sklearn.tree import _tree
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedKFold

In [3]:
import pydotplus
from sklearn.externals.six import StringIO 
from IPython.display import Image

/home/mwei/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [4]:
import pydotplus
from sklearn.externals.six import StringIO 
from IPython.display import Image

In [5]:
# card sheets

card_ol = pd.read_csv('Card_processed/card_ol.csv',encoding='GBK').drop('ol_neg',axis=1)
card_ol = mm.reduce_mem_usage(card_ol)

card_gene = pd.read_csv('Card_processed/card_gene.csv',encoding='GBK')
card_gene = mm.reduce_mem_usage(card_gene)
zero_sum = (card_gene.iloc[:,1:]==0).sum()
cond = (zero_sum/card_gene.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
card_gene = card_gene[['id']+kept]

card_volatility = pd.read_csv('Card_processed/card_volatility.csv',encoding='GBK')
card_volatility = mm.reduce_mem_usage(card_volatility)
zero_sum = (card_volatility.iloc[:,1:]==0).sum()
cond = (zero_sum/card_volatility.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
card_volatility = card_volatility[['id']+kept]

card_window = pd.read_csv('Card_processed/card_window.csv',encoding='GBK')
card_window = mm.reduce_mem_usage(card_window)

card_cross1 = pd.read_csv('Card_processed/card_cross1.csv',encoding='GBK')
card_cross1 = mm.reduce_mem_usage(card_cross1)
zero_sum = (card_cross1.iloc[:,1:]==0).sum()
cond = (zero_sum/card_cross1.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
card_cross1 = card_cross1[['id']+kept]

card_cross2 = pd.read_csv('Card_processed/card_cross2.csv',encoding='GBK')
card_cross2 = mm.reduce_mem_usage(card_cross2)
zero_sum = (card_cross2.iloc[:,1:]==0).sum()
cond = (zero_sum/card_cross2.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
card_cross2 = card_cross2[['id']+kept]

card_cross3 = pd.read_csv('Card_processed/card_cross3.csv',encoding='GBK')
card_cross3 = mm.reduce_mem_usage(card_cross3)
zero_sum = (card_cross3.iloc[:,1:]==0).sum()
cond = (zero_sum/card_cross3.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
card_cross3 = card_cross3[['id']+kept]

card_cross4 = pd.read_csv('Card_processed/card_cross4.csv',encoding='GBK')
card_cross4 = mm.reduce_mem_usage(card_cross4)
zero_sum = (card_cross4.iloc[:,1:]==0).sum()
cond = (zero_sum/card_cross4.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
card_cross4 = card_cross4[['id']+kept]

Reduce Memory Usage Function Reports:
Memory usage of dataframe is 80.06 MB
Memory usage after optimization is: 18.90 MB
Decreased by 76.4%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 169.02 MB
Memory usage after optimization is: 60.05 MB
Decreased by 64.5%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 391.41 MB
Memory usage after optimization is: 186.81 MB
Decreased by 52.3%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 44.48 MB
Memory usage after optimization is: 13.34 MB
Decreased by 70.0%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 329.14 MB
Memory usage after optimization is: 128.99 MB
Decreased by 60.8%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 1076.36 MB
Memory usage after optimization is: 369.17 MB
Decreased by 65.7%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 649.38 MB
Memory usage after optimization is: 251.30 MB
Decreased by 61.3

In [6]:
card = pd.merge(card_ol,card_gene,on='id',how='inner')
card = pd.merge(card,card_window,on='id',how='inner')
card = pd.merge(card,card_volatility,on='id',how='inner')
card = pd.merge(card,card_cross1,on='id',how='inner')
card = pd.merge(card,card_cross2,on='id',how='inner')
card = pd.merge(card,card_cross3,on='id',how='inner')
card = pd.merge(card,card_cross4,on='id',how='inner')
del card_ol,card_window,card_volatility,card_gene,card_cross1,card_cross2,card_cross3,card_cross4

In [7]:
# loan sheets

loan_ol = pd.read_csv('Loan_processed/loan_ol.csv',encoding='GBK').drop('ol_neg',axis=1)
loan_ol = mm.reduce_mem_usage(loan_ol)

loan_gene = pd.read_csv('Loan_processed/loan_gene.csv',encoding='GBK')
loan_gene = mm.reduce_mem_usage(loan_gene)
zero_sum = (loan_gene.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_gene.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_gene = loan_gene[['id']+kept]

loan_volatility = pd.read_csv('Loan_processed/loan_volatility.csv',encoding='GBK')
loan_volatility = mm.reduce_mem_usage(loan_volatility)
zero_sum = (loan_volatility.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_volatility.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_volatility = loan_volatility[['id']+kept]

loan_window = pd.read_csv('Loan_processed/loan_window.csv',encoding='GBK')
loan_window = mm.reduce_mem_usage(loan_window)

loan_cross1 = pd.read_csv('Loan_processed/loan_cross1.csv',encoding='GBK')
loan_cross1 = mm.reduce_mem_usage(loan_cross1)
zero_sum = (loan_cross1.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_cross1.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_cross1 = loan_cross1[['id']+kept]

loan_cross2 = pd.read_csv('Loan_processed/loan_cross2.csv',encoding='GBK')
loan_cross2 = mm.reduce_mem_usage(loan_cross2)
zero_sum = (loan_cross2.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_cross2.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_cross2 = loan_cross2[['id']+kept]

loan_cross3 = pd.read_csv('Loan_processed/loan_cross3.csv',encoding='GBK')
loan_cross3 = mm.reduce_mem_usage(loan_cross3)
zero_sum = (loan_cross3.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_cross3.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_cross3 = loan_cross3[['id']+kept]

loan_cross4 = pd.read_csv('Loan_processed/loan_cross4.csv',encoding='GBK')
loan_cross4 = mm.reduce_mem_usage(loan_cross4)
zero_sum = (loan_cross4.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_cross4.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_cross4 = loan_cross4[['id']+kept]

loan_cross5 = pd.read_csv('Loan_processed/loan_cross5.csv',encoding='GBK')
loan_cross5 = mm.reduce_mem_usage(loan_cross5)
zero_sum = (loan_cross5.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_cross5.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_cross5 = loan_cross5[['id']+kept]

loan_cross6 = pd.read_csv('Loan_processed/loan_cross6.csv',encoding='GBK')
loan_cross6 = mm.reduce_mem_usage(loan_cross6)
zero_sum = (loan_cross6.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_cross6.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_cross6 = loan_cross6[['id']+kept]

Reduce Memory Usage Function Reports:
Memory usage of dataframe is 59.74 MB
Memory usage after optimization is: 16.60 MB
Decreased by 72.2%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 132.76 MB
Memory usage after optimization is: 47.30 MB
Decreased by 64.4%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 238.97 MB
Memory usage after optimization is: 112.85 MB
Decreased by 52.8%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 33.19 MB
Memory usage after optimization is: 9.96 MB
Decreased by 70.0%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 723.54 MB
Memory usage after optimization is: 275.48 MB
Decreased by 61.9%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 544.31 MB
Memory usage after optimization is: 220.71 MB
Decreased by 59.5%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 245.60 MB
Memory usage after optimization is: 87.95 MB
Decreased by 64.2%
 

In [8]:
loan = pd.merge(loan_ol,loan_gene,on='id',how='inner')
loan = pd.merge(loan,loan_window,on='id',how='inner')
loan = pd.merge(loan,loan_volatility,on='id',how='inner')
loan = pd.merge(loan,loan_cross1,on='id',how='inner')
loan = pd.merge(loan,loan_cross2,on='id',how='inner')
loan = pd.merge(loan,loan_cross3,on='id',how='inner')
loan = pd.merge(loan,loan_cross4,on='id',how='inner')
loan = pd.merge(loan,loan_cross5,on='id',how='inner')
loan = pd.merge(loan,loan_cross6,on='id',how='inner')
del loan_ol,loan_window,loan_volatility,loan_gene,loan_cross1,loan_cross2,loan_cross3,loan_cross4,loan_cross5,loan_cross6

In [9]:
loan.shape,card.shape

((870050, 59), (1165961, 69))

In [10]:
# Jiben Sheet

jiben_dys = pd.read_csv('Jiben_processed/jiben_processed_dys0429.csv')
jiben_dys = mm.reduce_mem_usage(jiben_dys)

jiben_hy = pd.read_csv('Jiben_processed/jiben_processed_hy0429.csv')
jiben_hy = mm.reduce_mem_usage(jiben_hy)

jiben_yjy = pd.read_csv('Jiben_processed/yjy_jiben.csv')
jiben_yjy = mm.reduce_mem_usage(jiben_yjy)

observe_date = pd.read_csv('refer/jiben_vars_new_20200424.csv',skiprows=list(range(779441,779509)),
                           encoding="ISO-8859-1",error_bad_lines=False,low_memory=False)[['id','nasrdw_recd_date']]
observe_date = mm.reduce_mem_usage(observe_date)

Reduce Memory Usage Function Reports:
Memory usage of dataframe is 1460.67 MB
Memory usage after optimization is: 226.04 MB
Decreased by 84.5%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 461.43 MB
Memory usage after optimization is: 104.44 MB
Decreased by 77.4%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 174.59 MB
Memory usage after optimization is: 48.33 MB
Decreased by 72.3%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 24.94 MB
Memory usage after optimization is: 24.94 MB
Decreased by 0.0%
 


In [11]:
jiben = pd.merge(jiben_dys,jiben_hy,how='inner',on='id')
jiben = pd.merge(jiben,jiben_yjy,how='inner',on='id')
jiben = pd.merge(observe_date,jiben,how='inner',on='id')

jiben.drop(['age18~21','age21~24','age24~28','age28~49','age50'],axis=1,inplace=True)

In [12]:
del jiben_dys,jiben_hy,jiben_yjy,observe_date

In [57]:
# zero_sum = (jiben.iloc[:,2:]==0).sum()
# cond = (zero_sum/jiben.shape[0])<=0.7
# kept = list(zero_sum.loc[cond].index.values)
# jiben = jiben[['id','bad']+kept]

**Add New Features**

In [24]:
loan_3_sum = pd.read_csv('Loan_processed/Record3_sumamounts.csv')
loan_3_sum = mm.reduce_mem_usage(loan_3_sum)
zero_sum = (loan_3_sum.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_3_sum.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_3_sum = loan_3_sum[['id']+kept]

loan_6_sum = pd.read_csv('Loan_processed/Record6_sumamounts.csv')
loan_6_sum = mm.reduce_mem_usage(loan_6_sum)
zero_sum = (loan_6_sum.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_6_sum.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_6_sum = loan_6_sum[['id']+kept]

loan_latest = pd.read_csv('Loan_processed/loan_latest.csv')
loan_latest = mm.reduce_mem_usage(loan_latest)
zero_sum = (loan_latest.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_latest.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_latest = loan_latest[['id']+kept]

loan_latest24 = pd.read_csv('Loan_processed/loan_24_derive.csv')
loan_latest24 = mm.reduce_mem_usage(loan_latest24)
zero_sum = (loan_latest24.iloc[:,1:]==0).sum()
cond = (zero_sum/loan_latest24.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
loan_latest24 = loan_latest24[['id']+kept]

card_latest24 = pd.read_csv('Card_processed/card_24_derive.csv')
card_latest24 = mm.reduce_mem_usage(card_latest24)
zero_sum = (card_latest24.iloc[:,1:]==0).sum()
cond = (zero_sum/card_latest24.shape[0])<=0.6
kept = list(zero_sum.loc[cond].index.values)
card_latest24 = card_latest24[['id']+kept]

Reduce Memory Usage Function Reports:
Memory usage of dataframe is 57.85 MB
Memory usage after optimization is: 31.82 MB
Decreased by 45.0%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 57.85 MB
Memory usage after optimization is: 31.82 MB
Decreased by 45.0%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 69.42 MB
Memory usage after optimization is: 39.05 MB
Decreased by 43.7%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 110.67 MB
Memory usage after optimization is: 23.34 MB
Decreased by 78.9%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 138.98 MB
Memory usage after optimization is: 31.27 MB
Decreased by 77.5%
 


**Big Sheet**

In [25]:
big_sheet = pd.merge(jiben,card,on='id',how='left')
big_sheet = pd.merge(big_sheet,loan,on='id',how='left')
big_sheet = pd.merge(big_sheet,loan_3_sum,on='id',how='left')
big_sheet = pd.merge(big_sheet,loan_6_sum,on='id',how='left')
big_sheet = pd.merge(big_sheet,loan_latest,on='id',how='left')
big_sheet = pd.merge(big_sheet,loan_latest24,on='id',how='left')
big_sheet = pd.merge(big_sheet,card_latest24,on='id',how='left')
big_sheet = mm.reduce_mem_usage(big_sheet)

Reduce Memory Usage Function Reports:
Memory usage of dataframe is 1664.88 MB
Memory usage after optimization is: 1190.98 MB
Decreased by 28.5%
 


In [26]:
big_sheet.drop('opendate2',axis=1,inplace=True)
big_sheet.drop('index',axis=1,inplace=True)

In [27]:
del loan_3_sum,loan_6_sum,loan_latest,loan_latest24,card_latest24

In [28]:
big_sheet = big_sheet.replace([np.inf,-np.inf],np.nan)
big_sheet.fillna(0,inplace=True)

In [29]:
big_sheet.shape

(1634600, 309)

In [64]:
big_sheet.head()

,id,nasrdw_recd_date,bad,age_na,age_ol,var_jb_1_18~21,var_jb_1_21~24,var_jb_1_24~28,var_jb_1_28~49,var_jb_1_50,...,scheduledpaymentamount,actualpaymentamount,loan_24_*,loan_24_/,loan_24_N,loan_24_nan,card_24_*,card_24_/,card_24_nan,card_24_N
0,56a13c06ee2f2a0db827d0d9450213d3,20171220,G,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,38.0,0.0,5.0,34.0
1,c6342448d324d0e99e0d89175a2da335,20171229,G,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,82e5383d3b794f2555dadb7e37e9108e,20180208,G,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,a5e91382fc770d3bf1c3845d113d85a5,20180306,G,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,57e97bcbc4090d25d1c783fcb071b6df,20180314,G,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
del jiben,card,loan

In [31]:
cond1 = big_sheet['bad']=='N'
cond2 = big_sheet['bad']=='R'
cond3 = big_sheet['bad']=='G'
cond4 = big_sheet['bad']=='B'

cond_1 = cond3|cond4
cond_2 = cond1

big_sheet_N = big_sheet.loc[cond_2].reset_index(drop=True)
big_sheet_GB = big_sheet.loc[cond_1].reset_index(drop=True)

In [32]:
cond2017 = big_sheet_GB['nasrdw_recd_date']<'20180101'
cond201801 = ((big_sheet_GB['nasrdw_recd_date']>'20171231')&(big_sheet_GB['nasrdw_recd_date']<'20180201'))
cond201802 = ((big_sheet_GB['nasrdw_recd_date']>'20180131')&(big_sheet_GB['nasrdw_recd_date']<'20180301'))
cond201803 = ((big_sheet_GB['nasrdw_recd_date']>'20180228')&(big_sheet_GB['nasrdw_recd_date']<'20180401'))
cond201804 = ((big_sheet_GB['nasrdw_recd_date']>'20180331')&(big_sheet_GB['nasrdw_recd_date']<'20180501'))
cond201805 = ((big_sheet_GB['nasrdw_recd_date']>'20180430')&(big_sheet_GB['nasrdw_recd_date']<'20180601'))
cond201806 = ((big_sheet_GB['nasrdw_recd_date']>'20180531')&(big_sheet_GB['nasrdw_recd_date']<'20180701'))
cond201807 = ((big_sheet_GB['nasrdw_recd_date']>'20180630')&(big_sheet_GB['nasrdw_recd_date']<'20180801'))
cond201808 = ((big_sheet_GB['nasrdw_recd_date']>'20180731')&(big_sheet_GB['nasrdw_recd_date']<'20180901'))
cond201809 = ((big_sheet_GB['nasrdw_recd_date']>'20180831')&(big_sheet_GB['nasrdw_recd_date']<'20181001'))
cond201810 = ((big_sheet_GB['nasrdw_recd_date']>'20180930')&(big_sheet_GB['nasrdw_recd_date']<'20181101'))
cond201811 = ((big_sheet_GB['nasrdw_recd_date']>'20181031')&(big_sheet_GB['nasrdw_recd_date']<'20181201'))
cond201812 = ((big_sheet_GB['nasrdw_recd_date']>'20181130')&(big_sheet_GB['nasrdw_recd_date']<'20190101'))

In [33]:
big_sheet_GB['month_mark'] = cond2017*2017
big_sheet_GB['month_mark'].loc[cond201801] = 1
big_sheet_GB['month_mark'].loc[cond201802] = 2
big_sheet_GB['month_mark'].loc[cond201803] = 3
big_sheet_GB['month_mark'].loc[cond201804] = 4
big_sheet_GB['month_mark'].loc[cond201805] = 5
big_sheet_GB['month_mark'].loc[cond201806] = 6
big_sheet_GB['month_mark'].loc[cond201807] = 7
big_sheet_GB['month_mark'].loc[cond201808] = 8
big_sheet_GB['month_mark'].loc[cond201809] = 9
big_sheet_GB['month_mark'].loc[cond201810] = 10
big_sheet_GB['month_mark'].loc[cond201811] = 11
big_sheet_GB['month_mark'].loc[cond201812] = 12

/home/mwei/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [34]:
big_sheet_GB['month_mark'].unique()

array([2017,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
         12,    1])

In [35]:
big_sheet_GB.drop('nasrdw_recd_date',axis=1,inplace=True)
big_sheet_N.drop('nasrdw_recd_date',axis=1,inplace=True)

In [90]:
big_sheet_GB['bad'].value_counts()

G    888245
B     96956
Name: bad, dtype: int64

In [91]:
big_sheet_N.shape

(450609, 308)

**Outside Data**

In [ ]:
outside_data = pd.read_csv('outside_data/outside_data_0430.csv',encoding='GBK')
outside_data = mm.reduce_mem_usage(outside_data)

In [ ]:
outside_data.shape

In [ ]:
outside_data.drop(['公积金数据','养老金数据','资产表数据'],axis=1,inplace=True)

## 树分群

In [ ]:
class TreeSplit():
    
    def __init__(self,df,depths,min_leaf):
        
        self.df = df.copy()
        self.depths = depths
        self.min_leaf = min_leaf

    def splitGB_NR(self):
        
        cond1 = self.df['bad']=='N'
        cond2 = self.df['bad']=='R'
        cond3 = self.df['bad']=='G'
        cond4 = self.df['bad']=='B'

        cond_1 = cond3|cond4
        cond_2 = cond1

        self.df_NR = self.df.loc[cond_2].reset_index(drop=True)
        self.df_GB = self.df.loc[cond_1].reset_index(drop=True)
        
        self.df_GB['bad'].replace({'R':'B'},inplace=True)

        return self.df_GB,self.df_NR
    
    def XY_split(self):
        
        self.X_GB = self.df_GB.drop(['id','bad'],axis=1)
        self.y_GB = self.df_GB['bad']
        
        self.X_NR = self.df_NR.drop(['id','bad'],axis=1)
        
    def FitTree(self):
        
        print('start to split by tree')
        start = time.time()
        tree_clf = DecisionTreeClassifier(random_state=2,max_depth=self.depths,
                                          min_samples_leaf=self.min_leaf)
        
        tree_clf.fit(self.X_GB,self.y_GB)
        self.tree_clf = tree_clf
        
        self.SubsetID_NR = tree_clf.apply(self.X_NR)
        
        self.grandson_split_ID_NR = np.unique(self.SubsetID_NR)
        end = time.time()
        print('tree split finished, time cost {}seconds'.format(np.round(end-start,2)))
        
    def Visualize(self):
        
        self.dot_data = StringIO()
        tree.export_graphviz(self.tree_clf, 
                             out_file=self.dot_data,
                             feature_names=self.X_GB.columns,
                             filled=True, 
                             rounded=True, 
                             special_characters=True)

        self.graph = pydotplus.graph_from_dot_data(self.dot_data.getvalue()) 
        
        
    def SubSetsRecord(self):
        
        print('start to get leaves\' info')
        start = time.time()
        dec_path = self.tree_clf.decision_path(self.X_GB)
        self.all_nodes = collections.defaultdict(list)

        for d, dec in enumerate(dec_path):
            for i in range(self.tree_clf.tree_.node_count):
                if dec.toarray()[0][i] == 1:
                    self.all_nodes[i].append(d) 
        
        self.NR_nodes = {}
        for i in self.grandson_split_ID_NR:
            self.NR_nodes[i] = np.where(self.SubsetID_NR==i)
        
        self.end_nodes = {}
        for i in self.grandson_split_ID_NR:
            self.end_nodes[i] = self.all_nodes[i]
                
        end = time.time()
        print('leaves\' info restored, time cost {}seconds'.format(np.round(end-start,2)))
            
    def ExtractRules(self):
        
        tree_ = self.tree_clf.tree_
        self.feature_name = [self.feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
                             for i in tree_.feature]

        #print("def tree({}):".format(", ".join(feature_names)))

        def recurse(node, depth):
            indent = "  " * depth
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                self.split_name = self.feature_name[node]
                self.threshold = tree_.threshold[node]

#                 print('name',name)
#                 print('threshold',threshold)

                print("{}if {} <= {}:".format(indent, self.split_name, self.threshold))
                recurse(tree_.children_left[node], depth + 1)
                print("{}else:  # if {} > {}".format(indent, self.split_name, self.threshold))
                recurse(tree_.children_right[node], depth + 1)
            else:
                print("{}return {}".format(indent, tree_.value[node]))

        recurse(0, 1)
    
    
    def Ignite(self):
        
        self.splitGB_NR()
        self.XY_split()
        self.FitTree()
        self.Visualize()
        self.SubSetsRecord()

**执行树分群**

In [ ]:
outside_data_tree_dict = {}
for i in [3,4,5]:
    outside_data_tree = TreeSplit(outside_data,i,1000)
    outside_data_tree.Ignite()
    outside_data_tree_dict[i] = outside_data_tree
outside_data_tree3 = outside_data_tree_dict[3]
outside_data_tree4 = outside_data_tree_dict[4]
outside_data_tree5 = outside_data_tree_dict[5]
del outside_data_tree_dict,outside_data_tree

In [ ]:
outside_data_tree3 = TreeSplit(outside_data,3,100000)
outside_data_tree3.Ignite()

In [ ]:
Image(outside_data_tree3.graph.create_png())

In [ ]:
Image(outside_data_tree4.graph.create_png())

In [ ]:
Image(outside_data_tree5.graph.create_png())

**LR**

In [97]:
lr_clf0 = LogisticRegression()
lr_clf1 = LogisticRegression(max_iter=1000)
lr_clf2 = LogisticRegression(max_iter=1000,C=0.1)


def LR(lr_clf,X_GB,y_GB,X_N_df,i,return_dict,if_month=1):
    
    all_fts = set(X_GB.columns)
    ft_stat = (X_GB != 0).sum() 
    cond = (ft_stat==0)
    not_used = set(ft_stat.loc[cond].index.values)
    used_fts = list(all_fts-not_used)
    
    start = time.time()
    
    if if_month==1:
        scaler = MinMaxScaler()
        X_GB.iloc[:,:-1] = scaler.fit_transform(X_GB.iloc[:,:-1])
        cond = X_GB['month_mark'].isin(month_pairs[i]['train'])
        X_train = X_GB.loc[cond].drop('month_mark',axis=1).reset_index(drop=True)
        y_train = y_GB.loc[cond]
        
        cond = X_GB['month_mark'].isin(month_pairs[i]['test'])
        X_test = X_GB.loc[cond].drop('month_mark',axis=1).reset_index(drop=True)
        y_test = y_GB.loc[cond]        
        
    else:
        scaler = MinMaxScaler()
        X_GB_scaled = scaler.fit_transform(X_GB)
        X_train,X_test,y_train,y_test = train_test_split(X_GB_scaled,y_GB,test_size=0.3,random_state=2)
    
    scaler = MinMaxScaler()    
    X_N_scaled = scaler.fit_transform(X_N_df.drop(['id','bad'],axis=1))
    
    train_good = sum(y_train)
    train_bad = len(y_train)-train_good

    test_good = sum(y_test)
    test_bad = len(y_test)-test_good
    
    total_good = train_good + test_good  
    total_bad = train_bad + test_bad
        
    print('start to train and estimate model{}'.format(i))
    print()
    lr_clf.fit(X_train,y_train)
    
    pred = lr_clf.predict_proba(X_test)
    pred_train = lr_clf.predict_proba(X_train)
    
    auc = roc_auc_score(y_test,pred[:,1])
    auc_train = roc_auc_score(y_train,pred_train[:,1])
    
    fpr,tpr,thresholds = roc_curve(y_test,pred[:,1])
    ks = max(tpr-fpr)
    
    fpr,tpr,thresholds = roc_curve(y_train,pred_train[:,1])
    ks_train = max(tpr-fpr)
    
    NR_pred = lr_clf.predict_proba(X_N_scaled)[:,1]
    NR_pred_df = pd.DataFrame({'id':X_N_df['id'],'pred_good_prob':NR_pred})
    NR_pred_df['split'] = i
    
    coefs = lr_clf.coef_[0]
#     del lr_clf
    
    return_dict[i] = [auc,ks,y_test,pred[:,1],auc_train,ks_train,y_train,pred_train[:,1],used_fts,
                      train_good,train_bad,test_good,test_bad,total_good,total_bad,coefs,NR_pred_df,lr_clf]
    end = time.time()
    print('model {} finished, time spent {}seconds'.format(i,np.round(end-start,2)))
    

    
    return [auc,ks,y_test,pred[:,1],auc_train,ks_train,y_train,pred_train[:,1],
            used_fts,train_good,train_bad,test_good,test_bad,total_good,total_bad,
            coefs,NR_pred_df,lr_clf]

**LGBM**

In [36]:
lgbm_clf0 = lgb.LGBMClassifier()

lgbm_clf0_1 = lgb.LGBMClassifier(n_estimators=40,max_depth=5,lambda_l1=0.5,lambda_l2=5)

lgbm_clf1 = lgb.LGBMClassifier(max_bin=150,n_estimators=500,min_sum_hessian_in_leaf=1,
                             learning_rate=0.01,bagging_fraction = 0.8,objective='binary',
                             colsample_bytree = 0.7,feature_fraction = 0.8,
                             lambda_l1 = 0.5,lambda_l2 = 4.0,max_depth =5,importance_type='gain',
                             subsample = 0.7,silent=True,min_child_samples = 50)  

lgbm_clf2 = lgb.LGBMClassifier(max_bin=130,n_estimators=500,min_sum_hessian_in_leaf=1,
                             learning_rate=0.01,bagging_fraction = 0.7,objective='binary',
                             colsample_bytree = 0.7,feature_fraction = 0.7,
                             lambda_l1 = 1.2,lambda_l2 = 7.0,max_depth =5,importance_type='gain',
                             subsample = 0.75,silent=True,min_child_samples = 60) 

lgbm_clf3 = lgb.LGBMClassifier(max_bin=110,n_estimators=300,min_sum_hessian_in_leaf=1,
                             learning_rate=0.01,bagging_fraction = 0.7,objective='binary',
                             colsample_bytree = 0.7,feature_fraction = 0.7,
                             lambda_l1 = 4.0,lambda_l2 = 9.0,max_depth =5,importance_type='gain',
                             subsample = 0.65,silent=True,min_child_samples = 60) 

lgbm_clf4 = lgb.LGBMClassifier(max_bin=80,n_estimators=250,min_sum_hessian_in_leaf=1,
                             learning_rate=0.01,bagging_fraction = 0.7,objective='binary',
                             colsample_bytree = 0.6,feature_fraction = 0.7,
                             lambda_l1 = 6.0,lambda_l2 = 11.0,max_depth =4,importance_type='gain',
                             subsample = 0.65,silent=True,min_child_samples = 60) 

lgbm_clf5 = lgb.LGBMClassifier(max_bin=80,n_estimators=180,min_sum_hessian_in_leaf=1,
                             learning_rate=0.01,bagging_fraction = 0.7,objective='binary',
                             colsample_bytree = 0.6,feature_fraction = 0.7,
                             lambda_l1 = 6.0,lambda_l2 = 12.0,max_depth =4,importance_type='gain',
                             subsample = 0.6,silent=True,min_child_samples = 60) 

lgbm_clf6 = lgb.LGBMClassifier(max_bin=40,n_estimators=100,min_sum_hessian_in_leaf=1,
                             learning_rate=0.01,bagging_fraction = 0.7,objective='binary',
                             colsample_bytree = 0.5,feature_fraction = 0.6,
                             lambda_l1 = 8.0,lambda_l2 = 13.0,max_depth =4,importance_type='gain',
                             subsample = 0.5,silent=True,min_child_samples = 70) 

def LGBM(lgbm_clf,X_GB,y_GB,X_N_df,i,if_month=1):
    
    start = time.time()
    print('start to train LGBM model{}'.format(i))
    sk = StratifiedKFold(n_splits=5, shuffle=True)
    
    if if_month==1:
        scaler = MinMaxScaler()
        X_GB.iloc[:,:-1] = scaler.fit_transform(X_GB.iloc[:,:-1])
        cond = X_GB['month_mark'].isin(month_pairs[i]['train'])
        X_train = X_GB.loc[cond].drop('month_mark',axis=1).values
        Y_train = y_GB.loc[cond]
        
        cond = X_GB['month_mark'].isin(month_pairs[i]['test'])
        X_test = X_GB.loc[cond].drop('month_mark',axis=1).values
        Y_test = y_GB.loc[cond] 
    else:
        scaler = MinMaxScaler()

        X_GB_scaled = scaler.fit_transform(X_GB)
        X_train,X_test,Y_train,Y_test = train_test_split(X_GB_scaled,y_GB,test_size=0.3,random_state=2)

    scaler = MinMaxScaler()    
    X_N_scaled = scaler.fit_transform(X_N_df.drop(['id','bad'],axis=1))
    
    train_good = sum(Y_train)
    train_bad = len(Y_train)-train_good

    test_good = sum(Y_test)
    test_bad = len(Y_test)-test_good
    
    total_good = train_good + test_good  
    total_bad = train_bad + test_bad  
        
    preds = []
    preds_train = []   
    
    NR_preds = []
    
    ftimps = []
    
    for train_index, val_index in sk.split(X_train, Y_train):

        x_train, x_val = X_train[train_index,:], X_train[val_index,:]
        y_train, y_val = Y_train.iloc[train_index], Y_train.iloc[val_index]

        lgbm_clf.fit(x_train, y_train, eval_set=[(x_val, y_val)],early_stopping_rounds=20)

        preds.append(lgbm_clf.predict_proba(X_test)[:,1])
        preds_train.append(lgbm_clf.predict_proba(X_train)[:,1])
        
        NR_preds.append(lgbm_clf.predict_proba(X_N_scaled)[:,1])
        
        ftimps.append(lgbm_clf.feature_importances_)

    pred = np.mean(preds,axis=0)
    pred_train = np.mean(preds_train,axis=0)
        
    NR_pred = np.mean(NR_preds,axis=0)
    
    ftimp = np.mean(ftimps,axis=0)
    
    auc = roc_auc_score(Y_test,pred)
    auc_train = roc_auc_score(Y_train,pred_train)
    
    fpr,tpr,thresholds= roc_curve(Y_test,pred)
    ks = max(tpr-fpr)
    GB_thres = 1 - thresholds[np.argmax(tpr-fpr)]
    
    fpr,tpr,thresholds= roc_curve(Y_train,pred_train)
    ks_train = max(tpr-fpr)
 
    NR_pred_df = pd.DataFrame({'id':X_N_df['id'],'pred_good_prob':NR_pred})
    NR_pred_df['split'] = i
    
    end = time.time()
    print('LGBM model{} finished, time spent {}seconds'.format(i,np.round(end-start,2)))
    print()
    
#     del lgbm_clf
    
    return [auc,ks,Y_test,pred,auc_train,ks_train,
            Y_train,pred_train,train_good,train_bad,
            test_good,test_bad,total_good,total_bad,
            ftimp,NR_pred_df,GB_thres,lgbm_clf]

**XGB**

In [ ]:
xgb_clf0 = xgb.XGBClassifier()

xgb_clf1 = xgb.XGBClassifier(colsample_bytree=0.7, gamma=0,importance_type='gain',
           eta=0.01, max_depth=6,min_child_weight=1, 
           n_estimators=500,objective='binary:logistic', random_state=2, 
           reg_alpha=0.3,reg_lambda=1,subsample=0.7)

xgb_clf2 = xgb.XGBClassifier(colsample_bytree=0.7, gamma=1,importance_type='gain',
           eta=0.01, max_depth=5,min_child_weight=1, 
           n_estimators=400,objective='binary:logistic', random_state=2, 
           reg_alpha=2,reg_lambda=1,subsample=0.6)

def XGB(X_GB,y_GB,X_N_df,i,xgb_clf):
    
    start = time.time()
    print('start to train XGB model{}'.format(i))
    scaler = MinMaxScaler()
    X_GB_scaled = scaler.fit_transform(X_GB)
    X_train,X_test,y_train,y_test = train_test_split(X_GB_scaled,y_GB,test_size=0.3,random_state=2)
    
    scaler = MinMaxScaler()    
    X_N_scaled = scaler.fit_transform(X_N_df.drop(['id','nasrdw_recd_date','bad'],axis=1))
    
    train_good = sum(y_train)
    train_bad = len(y_train)-train_good

    test_good = sum(y_test)
    test_bad = len(y_test)-test_good
    
    total_good = train_good + test_good  
    total_bad = train_bad + test_bad
    
    X_dmatrix = xgb.DMatrix(X_train,label=y_train)
#     xgb_clf = xgb.XGBClassifier(**params)
    xgb_clf.fit(X_train,y_train)
    
#     eval_set = [(X_test, y_test)]
#     xgb_clf.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=False)
    
    pred = xgb_clf.predict_proba(X_test)
    pred_train = xgb_clf.predict_proba(X_train)
    
    auc = roc_auc_score(y_test,pred[:,1])
    auc_train = roc_auc_score(y_train,pred_train[:,1])
    
    fpr,tpr,thresholds= roc_curve(y_test,pred[:,1])
    ks = max(tpr-fpr)
    fpr,tpr,thresholds= roc_curve(y_train,pred_train[:,1])
    ks_train = max(tpr-fpr)   
    
    NR_pred = xgb_clf.predict_proba(X_N_scaled)[:,1]
    NR_pred_df = pd.DataFrame({'id':X_N_df['id'],'pred_good_prob':NR_pred})
    NR_pred_df['split'] = i
    
#     return_dict[i] = xgb_clf,auc,ks,y_test,pred[:,1]
    end = time.time()
    print('model {} finished, time spent {}seconds'.format(i,np.round(end-start,2)))
    
    return xgb_clf,auc,ks,y_test,pred[:,1],auc_train,ks_train,y_train,pred_train[:,1],train_good,train_bad,test_good,test_bad,total_good,total_bad,xgb_clf.feature_importances_,NR_pred_df

**IV**

In [ ]:
# -*- coding: utf-8 -*-
class Discrete(object):
    def __init__(self, x, y, continue_list,dis_num):
        self.x = x
        self.y = y
        self.continue_list = continue_list
        self.dis_num = dis_num

# 判断特征数据是否为离散型
    def feature_discretion(self):
        X = self.x
        temp = np.array([])
        for i in range(0, X.shape[-1]):
            x1 = X.iloc[:, i]
            if x1.name in self.continue_list:
                x2 = self.Quantile_Discretiton(x1)
                temp = np.r_[temp, x2]
            else:
                temp = np.r_[temp, x1]
        return temp.reshape(X.T.shape).T

# 分位数分箱进行离散化
    def Quantile_Discretiton(self,X):
        res = np.array([0] * X.shape[-1], dtype=int)
        X = np.array([X])
        for i in range(self.dis_num):
            times = 100 // self.dis_num
            point1 = np.percentile(X, i * times)
            point2 = np.percentile(X, (i + 1) * times)
            index_s = np.where((X >= point1) & (X <= point2))[1]
            x1 = X.T[index_s]
            mask = np.in1d(X, x1)
            res[mask] = (i + 1)
        return res


class Feature_filter(Discrete):
    def __init__(self, X, Y, event, con_list,dis_num):
        self.X = X
        self.Y = Y
        self.event = event
        self.con_list = con_list
        self.dis_num = dis_num

# 结果筛选
    def Result(self, percent=0.5):
        X1 = Discrete(self.X,self.Y,self.con_list,self.dis_num).feature_discretion()
        iv = self.Woe_iv(X1)
        iv = iv.sort_values(ascending=False)
        method_per = int(float(percent * iv.shape[0])) + 1
        if method_per >= iv.shape[0]:
            res = iv
        else:
            res = iv[0:method_per]
        return res

# 求woe值和iv值
    def Woe_iv(self, XX):
        res_woe = np.array([])
        res_iv = np.array([])
        for i in range(0, XX.shape[-1]):
            X2 = XX[:, i]
            woe, iv = self.Woe_single_x(a=X2)
            res_woe = np.append(res_woe, woe)
            res_iv = np.append(res_iv, iv)
        res = pd.Series(res_iv, index=self.X.columns)
        return res

# 求单个特征的woe值
    def Woe_single_x(self, a):
        event_total, non_event_total = self.Count_binary(self.Y, self.event)
        x_labels = np.unique(a)
        woe_dict = {}
        iv = 0
        for x1 in x_labels:
            y1 = self.Y.iloc[np.where(a == x1)[0]]
            event_count, non_event_count = self.Count_binary(y1, self.event)
            rate_event = 1.0 * event_count / event_total
            rate_non_event = 1.0 * non_event_count / non_event_total
        # woe无穷大时处理
            if rate_event == 0:
                print()
            elif rate_non_event == 0:
                print()
            else:
                woe1 = math.log(rate_event / rate_non_event)
                woe_dict[x1] = woe1
                iv += (rate_event - rate_non_event) * woe1
        return woe_dict, iv

# 计算个数
    def Count_binary(self, b, event):
        event_count = (b == self.event).sum()
        non_event_count = b.shape[-1] - event_count
        return event_count, non_event_count




In [ ]:
def Run_IV(x,y_encode,IV_cols,return_dict,i):
    start = time.time()
    print('start to run iv on split{}'.format(i))
    result = Feature_filter(x,y_encode,1,IV_cols,5).Result(percent=1)
    return_dict[i]=result
    end=time.time()
    print('time cost {}seconds'.format(np.round(end-start,2)))

In [ ]:
Image(outside_data_tree3.graph.create_png())

**统一筛选IV**

In [ ]:
start = time.time()
print('start to run total iv')
total_iv = Feature_filter(big_sheet_GB.drop(['id','bad'],axis=1),
                          big_sheet_GB['bad'].replace({'G':1,'B':0}),
                          1,
                          list(big_sheet_GB.drop(['id','bad'],axis=1).columns),
                          5).Result(percent=1)
mother_iv_fts = list(total_iv.loc[total_iv>0].index.values)
end=time.time()
print('time cost {}seconds'.format(np.round(end-start,2)))

**LR训练模型**

In [ ]:
# model mother
mother_X = big_sheet_GB.drop(['id','bad'],axis=1)
mother_Y = big_sheet_GB['bad'].replace({'G':1,'B':0})
[mother_auc,mother_ks,mother_y_test,mother_pred,
 mother_auc_train,mother_ks_train,mother_y_train,mother_pred_train,
 mother_used_fts,mother_train_good,mother_train_bad,mother_test_good,
 mother_test_bad,mother_total_good,mother_total_bad,
 mother_coefs,mother_lr_NR_pred,mother_lr_clf] = LR(lr_clf0,mother_X,mother_Y,big_sheet_N,'mother',{})
mother_Y.value_counts(),mother_auc,mother_ks,mother_auc_train,mother_ks_train

In [ ]:
fpr,tpr,thresholds= roc_curve(mother_y_test,mother_pred)
thres = 1 - thresholds[np.argmax(tpr-fpr)]
thres

In [ ]:
sum(mother_lr_NR_pred['pred_good_prob']<(1-thres))

In [109]:
lr_clf0_0 = LogisticRegression()
X_GB = big_sheet_GB.drop(['id','bad'],axis=1)
Y_GB = big_sheet_GB['bad'].replace({'G':1,'B':0})
c = LR(lr_clf0_0,X_GB,Y_GB,big_sheet_N,'pair1',{},1)

In [37]:
# monthly recurrent
month_pairs = {'pair1':{'train':[1,2,3,4,5,6],'test':[10,11,12]},
               'pair2':{'train':[3,4,5,6,7,8],'test':[12,1,2]},
               'pair3':{'train':[5,6,7,8,9,10],'test':[2,3,4]},
               'pair4':{'train':[7,8,9,10,11,12],'test':[4,5,6]},
               'pair5':{'train':[9,10,11,12,1,2],'test':[6,7,8]},
               'pair6':{'train':[11,12,1,2,3,4],'test':[8,9,10]}}
# month_pair_keys = list(month_pairs.keys())
# if __name__ == '__main__':
#     manager = mp.Manager()
#     lr_month_recurrent_dict = manager.dict()
#     jobs = []
#     for i in month_pair_keys:
#         lr_clf0_0 = LogisticRegression()
#         X_GB = big_sheet_GB.drop(['id','bad'],axis=1)
#         Y_GB = big_sheet_GB['bad'].replace({'G':1,'B':0})
#         X_N = big_sheet_N
        
#         p = mp.Process(target=LR, args=(lr_clf0_0,X_GB,Y_GB,X_N,i,lr_month_recurrent_dict,1))
        
#         jobs.append(p)
#         p.start()

#     for proc in jobs:
#         proc.join()
        
# end = time.time()
# print('total time cost of duplicates processing takes {}seconds'.format(np.round(end-start,2)))

In [ ]:
start = time.time()
end_nodes = outside_data_tree3.end_nodes
NR_nodes = outside_data_tree3.NR_nodes

list1 = [11,10,7,14,13]
list2 = [6,4,3]

nodes_id = end_nodes.keys()
if __name__ == '__main__':
    manager = mp.Manager()
    lr3_return_dict = manager.dict()
    jobs = []
    for i in nodes_id:
        
        X_GB = big_sheet_GBR.iloc[end_nodes[i],:].drop(['id','bad'],axis=1)
        Y_GB = big_sheet_GBR['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0})
        X_NR = big_sheet_N.iloc[NR_nodes[i]]
        
        p = mp.Process(target=LR, args=(lr_clf1,X_GB,Y_GB,X_NR,i,lr3_return_dict))
        
#         if i in list1:
#             p = mp.Process(target=LR, args=(clf1,X_GB,Y_GB,X_i,lr3_return_dict))
#         else:
#             p = mp.Process(target=LR, args=(clf2,X_GB,Y_GB,i,lr3_return_dict))

        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
        
end = time.time()
print('total time cost of duplicates processing takes {}seconds'.format(np.round(end-start,2)))

In [ ]:
# [auc,ks,y_test,pred[:,1],auc_train,ks_train,y_train,pred_train[:,1],
#  used_fts,train_good,train_bad,test_good,test_bad,total_good,total_bad,
#  coefs,NR_pred_df]

auc,auc_train,i_s,ks,ks_train,num_samples = [],[],[],[],[],[]

train_good,train_bad,test_good,test_bad,total_good,total_bad = [],[],[],[],[],[]

used_fts = {}
coefs = {}
coefs['fts'] = mother_X.columns
coefs['mother']=mother_coefs
lr_preds_df = {}

for i in lr3_return_dict.keys():
    i_s.append(i)
    auc.append(lr3_return_dict[i][0])
    ks.append(lr3_return_dict[i][1])
    auc_train.append(lr3_return_dict[i][4])
    ks_train.append(lr3_return_dict[i][5])
    num_samples.append(len(outside_data_tree3.end_nodes[i]))
    train_good.append(lr3_return_dict[i][9])
    train_bad.append(lr3_return_dict[i][10])
    test_good.append(lr3_return_dict[i][11])
    test_bad.append(lr3_return_dict[i][12])
    total_good.append(lr3_return_dict[i][13])
    total_bad.append(lr3_return_dict[i][14])
    used_fts[i] = lr3_return_dict[i][8]
    coefs[i]=lr3_return_dict[i][15]
    lr_preds_df[i]=lr3_return_dict[i][-1]
    
i_s.append('mother')
auc.append(mother_auc)
ks.append(mother_ks)
auc_train.append(mother_auc_train)
ks_train.append(mother_ks_train)
num_samples.append(393294)
train_good.append(mother_train_good)
train_bad.append(mother_train_bad)
test_good.append(mother_test_good)
test_bad.append(mother_test_bad)
total_good.append(mother_total_good)
total_bad.append(mother_total_bad)
lr_preds_df['mother']=mother_lr_NR_pred

y_test_stack = np.array([])
y_pred_stack = np.array([])
y_pred_train_stack = np.array([])
y_train_stack = np.array([])
for i in lr3_return_dict.keys():
    y_test_stack = np.concatenate([y_test_stack,lr3_return_dict[i][2]])
    y_pred_stack = np.concatenate([y_pred_stack,lr3_return_dict[i][3]])
    y_pred_train_stack = np.concatenate([y_pred_train_stack,lr3_return_dict[i][7]])
    y_train_stack = np.concatenate([y_train_stack,lr3_return_dict[i][6]])
    
total_auc = roc_auc_score(y_test_stack,y_pred_stack)
fpr,tpr,thresholds= roc_curve(y_test_stack,y_pred_stack)
total_ks = max(tpr-fpr)

total_auc_train = roc_auc_score(y_train_stack,y_pred_train_stack)
fpr,tpr,thresholds= roc_curve(y_train_stack,y_pred_train_stack)
total_ks_train = max(tpr-fpr)

i_s.append('子群整合')
auc.append(total_auc)
ks.append(total_ks)
auc_train.append(total_auc_train)
ks_train.append(total_ks_train)
num_samples.append(393294)
train_good.append(mother_train_good)
train_bad.append(mother_train_bad)
test_good.append(mother_test_good)
test_bad.append(mother_test_bad)
total_good.append(mother_total_good)
total_bad.append(mother_total_bad)

total_auc,total_ks,total_auc_train,total_ks_train
lr3_result_df = pd.DataFrame({'子群ID':i_s,'test_good':test_good,'test_bad':test_bad,'test_auc':auc,'test_ks':ks,
                              'train_good':train_good,'train_bad':train_bad,'auc_train':auc_train,'ks_train':ks_train,
                              'total_good':total_good,'total_bad':total_bad,'num_sample':num_samples}).sort_values(by=['test_ks','test_auc'],ascending=False)

# lr3_480_0423_result_df.to_csv('lr3_result_df_480_0423.csv')
lr3_weights = pd.DataFrame(coefs)
# lr3_480_weights.to_csv('lr3_weights_df_480_0423.csv',encoding='GBK')

In [ ]:
lr3_result_df['train_test_gap'] = lr3_result_df['auc_train']-lr3_result_df['test_auc']
lr3_result_df.sort_values(by='train_test_gap')

In [ ]:
sum(lr_preds_df['mother']['pred_good_prob']<0.5)

In [ ]:
lr_preds_concated = pd.concat([lr_preds_df[3],lr_preds_df[4],lr_preds_df[10],lr_preds_df[11],
                               lr_preds_df[6],lr_preds_df[14],lr_preds_df[13],lr_preds_df[7]],axis=0)

In [ ]:
sum(lr_preds_concated['pred_good_prob']<0.5)

**LGBM**

In [ ]:
# xgb_clf,auc,ks,y_test,pred[:,1],auc_train,ks_train,y_train,
# pred_train[:,1],used_fts,train_good,train_bad,test_good,test_bad,total_good,total_bad
mother_X = big_sheet_GB.drop(['id','bad'],axis=1)
mother_Y = big_sheet_GB['bad'].replace({'G':1,'B':0})
mother_N = big_sheet_N
[lgbm_mother_auc,lgbm_mother_ks,lgbm_mother_y_test,lgbm_mother_pred,
 lgbm_mother_auc_train,lgbm_mother_ks_train,lgbm_mother_y_train,lgbm_mother_pred_train,
 lgbm_mother_train_good,lgbm_mother_train_bad,lgbm_mother_test_good,lgbm_mother_test_bad,
 lgbm_mother_total_good,lgbm_mother_total_bad,mother_lgbm_ftimp,
 lgbm_mother_NRpred,lgbm_mother_GBthres,mother_lgbm_clf] = LGBM(lgbm_clf0,mother_X,mother_Y,mother_N,'mother')
mother_Y.value_counts(),lgbm_mother_auc,lgbm_mother_ks,lgbm_mother_auc_train,lgbm_mother_ks_train

In [ ]:
sum(lgbm_mother_NRpred['pred_good_prob']<=(1-lgbm_mother_GBthres))

In [ ]:
lgbm_mother_NRpred.to_csv('lgbm_mother_0508.csv',index=False)

In [104]:
lgbm_recurrent_month = {}

X_GB = big_sheet_GB.drop(['id','bad'],axis=1)
Y_GB = big_sheet_GB['bad'].replace({'G':1,'B':0})

lgbm_clf0_0 = lgb.LGBMClassifier()
lgbm_recurrent_month['pair5'] = LGBM(lgbm_clf0_0,X_GB,Y_GB,big_sheet_N,'pair5',if_month=1)

# for i in month_pair_keys:
    
#     start = time.time()
    
#     lgbm_clf0_0 = lgb.LGBMClassifier()
#     lgbm_recurrent_month[i] = LGBM(lgbm_clf0_0,X_GB,Y_GB,big_sheet_N,i,if_month=1)
    
#     end = time.time()
#     print('{} model finished, time cost {}'.format(i,np.round(end-start,2)))

start to train LGBM modelpair5
[1]	valid_0's binary_logloss: 0.328847
Training until validation scores don't improve for 20 rounds
[2]	valid_0's binary_logloss: 0.322179
[3]	valid_0's binary_logloss: 0.316898
[4]	valid_0's binary_logloss: 0.312826
[5]	valid_0's binary_logloss: 0.309465
[6]	valid_0's binary_logloss: 0.306548
[7]	valid_0's binary_logloss: 0.303984
[8]	valid_0's binary_logloss: 0.301722
[9]	valid_0's binary_logloss: 0.29984
[10]	valid_0's binary_logloss: 0.298146
[11]	valid_0's binary_logloss: 0.296718
[12]	valid_0's binary_logloss: 0.29537
[13]	valid_0's binary_logloss: 0.294161
[14]	valid_0's binary_logloss: 0.29295
[15]	valid_0's binary_logloss: 0.291894
[16]	valid_0's binary_logloss: 0.290825
[17]	valid_0's binary_logloss: 0.289903
[18]	valid_0's binary_logloss: 0.289146
[19]	valid_0's binary_logloss: 0.28836
[20]	valid_0's binary_logloss: 0.287723
[21]	valid_0's binary_logloss: 0.28707
[22]	valid_0's binary_logloss: 0.286441
[23]	valid_0's binary_logloss: 0.28587
[24

[1]	valid_0's binary_logloss: 0.328885
Training until validation scores don't improve for 20 rounds
[2]	valid_0's binary_logloss: 0.322243
[3]	valid_0's binary_logloss: 0.317225
[4]	valid_0's binary_logloss: 0.313094
[5]	valid_0's binary_logloss: 0.309695
[6]	valid_0's binary_logloss: 0.306906
[7]	valid_0's binary_logloss: 0.304408
[8]	valid_0's binary_logloss: 0.302277
[9]	valid_0's binary_logloss: 0.300314
[10]	valid_0's binary_logloss: 0.298728
[11]	valid_0's binary_logloss: 0.297282
[12]	valid_0's binary_logloss: 0.295957
[13]	valid_0's binary_logloss: 0.29469
[14]	valid_0's binary_logloss: 0.293615
[15]	valid_0's binary_logloss: 0.292549
[16]	valid_0's binary_logloss: 0.291723
[17]	valid_0's binary_logloss: 0.290777
[18]	valid_0's binary_logloss: 0.29001
[19]	valid_0's binary_logloss: 0.289143
[20]	valid_0's binary_logloss: 0.288435
[21]	valid_0's binary_logloss: 0.287742
[22]	valid_0's binary_logloss: 0.28719
[23]	valid_0's binary_logloss: 0.286653
[24]	valid_0's binary_logloss: 

[1]	valid_0's binary_logloss: 0.328561
Training until validation scores don't improve for 20 rounds
[2]	valid_0's binary_logloss: 0.321721
[3]	valid_0's binary_logloss: 0.316447
[4]	valid_0's binary_logloss: 0.312214
[5]	valid_0's binary_logloss: 0.30869
[6]	valid_0's binary_logloss: 0.305876
[7]	valid_0's binary_logloss: 0.30341
[8]	valid_0's binary_logloss: 0.301211
[9]	valid_0's binary_logloss: 0.299243
[10]	valid_0's binary_logloss: 0.297599
[11]	valid_0's binary_logloss: 0.296069
[12]	valid_0's binary_logloss: 0.294727
[13]	valid_0's binary_logloss: 0.29352
[14]	valid_0's binary_logloss: 0.29229
[15]	valid_0's binary_logloss: 0.29108
[16]	valid_0's binary_logloss: 0.290104
[17]	valid_0's binary_logloss: 0.289178
[18]	valid_0's binary_logloss: 0.288342
[19]	valid_0's binary_logloss: 0.287522
[20]	valid_0's binary_logloss: 0.286837
[21]	valid_0's binary_logloss: 0.286155
[22]	valid_0's binary_logloss: 0.285443
[23]	valid_0's binary_logloss: 0.284853
[24]	valid_0's binary_logloss: 0.

In [105]:
lgbm_clf0_0 = lgb.LGBMClassifier()
lgbm_recurrent_month['pair6'] = LGBM(lgbm_clf0_0,X_GB,Y_GB,big_sheet_N,'pair6',if_month=1)

start to train LGBM modelpair6
[1]	valid_0's binary_logloss: 0.305964
Training until validation scores don't improve for 20 rounds
[2]	valid_0's binary_logloss: 0.297167
[3]	valid_0's binary_logloss: 0.290877
[4]	valid_0's binary_logloss: 0.285677
[5]	valid_0's binary_logloss: 0.281654
[6]	valid_0's binary_logloss: 0.278038
[7]	valid_0's binary_logloss: 0.27508
[8]	valid_0's binary_logloss: 0.272485
[9]	valid_0's binary_logloss: 0.270333
[10]	valid_0's binary_logloss: 0.26837
[11]	valid_0's binary_logloss: 0.266708
[12]	valid_0's binary_logloss: 0.265161
[13]	valid_0's binary_logloss: 0.263903
[14]	valid_0's binary_logloss: 0.262737
[15]	valid_0's binary_logloss: 0.261614
[16]	valid_0's binary_logloss: 0.26065
[17]	valid_0's binary_logloss: 0.259738
[18]	valid_0's binary_logloss: 0.258886
[19]	valid_0's binary_logloss: 0.258191
[20]	valid_0's binary_logloss: 0.257505
[21]	valid_0's binary_logloss: 0.25685
[22]	valid_0's binary_logloss: 0.25623
[23]	valid_0's binary_logloss: 0.25567
[24

[1]	valid_0's binary_logloss: 0.306061
Training until validation scores don't improve for 20 rounds
[2]	valid_0's binary_logloss: 0.297495
[3]	valid_0's binary_logloss: 0.291089
[4]	valid_0's binary_logloss: 0.286139
[5]	valid_0's binary_logloss: 0.282086
[6]	valid_0's binary_logloss: 0.278748
[7]	valid_0's binary_logloss: 0.275869
[8]	valid_0's binary_logloss: 0.273362
[9]	valid_0's binary_logloss: 0.271249
[10]	valid_0's binary_logloss: 0.269472
[11]	valid_0's binary_logloss: 0.267852
[12]	valid_0's binary_logloss: 0.266358
[13]	valid_0's binary_logloss: 0.265152
[14]	valid_0's binary_logloss: 0.264034
[15]	valid_0's binary_logloss: 0.263108
[16]	valid_0's binary_logloss: 0.262195
[17]	valid_0's binary_logloss: 0.261367
[18]	valid_0's binary_logloss: 0.260537
[19]	valid_0's binary_logloss: 0.259902
[20]	valid_0's binary_logloss: 0.259236
[21]	valid_0's binary_logloss: 0.258643
[22]	valid_0's binary_logloss: 0.258055
[23]	valid_0's binary_logloss: 0.257518
[24]	valid_0's binary_loglos

[1]	valid_0's binary_logloss: 0.306178
Training until validation scores don't improve for 20 rounds
[2]	valid_0's binary_logloss: 0.297502
[3]	valid_0's binary_logloss: 0.29099
[4]	valid_0's binary_logloss: 0.285978
[5]	valid_0's binary_logloss: 0.28181
[6]	valid_0's binary_logloss: 0.278266
[7]	valid_0's binary_logloss: 0.275277
[8]	valid_0's binary_logloss: 0.272748
[9]	valid_0's binary_logloss: 0.270449
[10]	valid_0's binary_logloss: 0.268485
[11]	valid_0's binary_logloss: 0.266836
[12]	valid_0's binary_logloss: 0.26529
[13]	valid_0's binary_logloss: 0.264022
[14]	valid_0's binary_logloss: 0.262823
[15]	valid_0's binary_logloss: 0.26175
[16]	valid_0's binary_logloss: 0.260812
[17]	valid_0's binary_logloss: 0.260009
[18]	valid_0's binary_logloss: 0.259154
[19]	valid_0's binary_logloss: 0.258341
[20]	valid_0's binary_logloss: 0.257713
[21]	valid_0's binary_logloss: 0.256994
[22]	valid_0's binary_logloss: 0.256382
[23]	valid_0's binary_logloss: 0.255852
[24]	valid_0's binary_logloss: 0

In [107]:
lgbm_recurrent_month['pair6'][0]

0.7821926189655073

In [ ]:
end_nodes = outside_data_tree3.end_nodes
nodes_id = end_nodes.keys()

NR_nodes = outside_data_tree3.NR_nodes

lgbm3_return_dict={}

for i in nodes_id:
    
    X_GB = big_sheet_GB.iloc[end_nodes[i],:].drop(['id','bad'],axis=1)
    Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0}) 
    X_N = big_sheet_N.iloc[NR_nodes[i]]
    
    lgbm3_return_dict[i] = LGBM(lgbm_clf0_1,X_GB,Y_GB,X_N,i)

In [ ]:
end_nodes = outside_data_tree5.end_nodes
nodes_id = end_nodes.keys()

NR_nodes = outside_data_tree5.NR_nodes

list1 = [27,25,34,35]
list2 = [28,24,31,7]
list3 = [11,16,32,40,15]
list4 = [45,48,5,44,4]
list5 = [38,8,41,12,18,47,19]

lgbm5_return_dict={}
for i in nodes_id:
    
    X_GB = big_sheet_GB.iloc[end_nodes[i],:].drop(['id','bad'],axis=1)
    Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0}) 
    X_NR = big_sheet_N.iloc[NR_nodes[i]]
    
    if i in list1:
        lgbm5_return_dict[i] = LGBM(lgbm_clf1,X_GB,Y_GB,X_NR,i)
        
    elif i in list2:
        lgbm5_return_dict[i] = LGBM(lgbm_clf2,X_GB,Y_GB,X_NR,i)
        
    elif i in list3:
        lgbm5_return_dict[i] = LGBM(lgbm_clf4,X_GB,Y_GB,X_NR,i)
        
    elif i in list4:
        lgbm5_return_dict[i] = LGBM(lgbm_clf4,X_GB,Y_GB,X_NR,i)
    
    else:
        lgbm5_return_dict[i] = LGBM(lgbm_clf4,X_GB,Y_GB,X_NR,i)
        

In [108]:
# [auc,ks,Y_test,pred,auc_train,ks_train,
# Y_train,pred_train,train_good,train_bad,test_good,test_bad,total_good,total_bad,
# ftimp,NR_pred_df，GB_thres]

# outside_data_tree
def MakeLGBMResult(lgbm_return_dict):
    
    auc,auc_train,i_s,ks,ks_train,num_samples,GB_thres = [],[],[],[],[],[],[]
    train_good,train_bad,test_good,test_bad,total_good,total_bad = [],[],[],[],[],[]

    ftimps = {}
#     ftimps['fts'] = mother_X.columns
#     ftimps['mother'] = mother_lgbm_ftimp
    
    lgbm_preds_dfs = {}
    
    lgbm_clfs = {}

    # 各子群结果
    for i in lgbm_return_dict.keys():

        i_s.append(i)
        auc.append(lgbm_return_dict[i][0])
        ks.append(lgbm_return_dict[i][1])
        auc_train.append(lgbm_return_dict[i][4])
        ks_train.append(lgbm_return_dict[i][5])
#         num_samples.append(len(outside_data_tree.end_nodes[i]))
        train_good.append(lgbm_return_dict[i][8])
        train_bad.append(lgbm_return_dict[i][9])
        test_good.append(lgbm_return_dict[i][10])
        test_bad.append(lgbm_return_dict[i][11])
        total_good.append(lgbm_return_dict[i][12])
        total_bad.append(lgbm_return_dict[i][13])
        GB_thres.append(lgbm_return_dict[i][-2])
        ftimps[i] = lgbm_return_dict[i][14]
        lgbm_preds_dfs[i] = lgbm_return_dict[i][-3]
        lgbm_clfs[i] = lgbm_return_dict[i][-1]
     
    # append母群结果
#     i_s.append('mother')
#     auc.append(lgbm_mother_auc)
#     ks.append(lgbm_mother_ks)
#     auc_train.append(lgbm_mother_auc_train)
#     ks_train.append(lgbm_mother_ks_train)
#     num_samples.append(393294)
#     train_good.append(lgbm_mother_train_good)
#     train_bad.append(lgbm_mother_train_bad)
#     test_good.append(lgbm_mother_test_good)
#     test_bad.append(lgbm_mother_test_bad)
#     total_good.append(lgbm_mother_total_good)
#     total_bad.append(lgbm_mother_total_bad)
#     GB_thres.append(lgbm_mother_GBthres)
#     lgbm_preds_dfs['mother'] = lgbm_mother_NRpred
    
    y_test_stack = np.array([])
    y_pred_stack = np.array([])
    y_pred_train_stack = np.array([])
    y_train_stack = np.array([])
    
    # 子群整合计算表现
    for i in lgbm_return_dict.keys():

        y_test_stack = np.concatenate([y_test_stack,lgbm_return_dict[i][2]])
        y_pred_stack = np.concatenate([y_pred_stack,lgbm_return_dict[i][3]])
        y_pred_train_stack = np.concatenate([y_pred_train_stack,lgbm_return_dict[i][7]])
        y_train_stack = np.concatenate([y_train_stack,lgbm_return_dict[i][6]])            
    
#     total_auc = roc_auc_score(y_test_stack,y_pred_stack)
#     fpr,tpr,thresholds= roc_curve(y_test_stack,y_pred_stack)
#     total_ks = max(tpr-fpr)
#     total_GBthres = 1 - thresholds[np.argmax(tpr-fpr)]

#     total_auc_train = roc_auc_score(y_train_stack,y_pred_train_stack)
#     fpr,tpr,thresholds= roc_curve(y_train_stack,y_pred_train_stack)
#     total_ks_train = max(tpr-fpr)

#     # append子群整合后的结果
#     i_s.append('子群整合')
#     auc.append(total_auc)
#     ks.append(total_ks)
#     auc_train.append(total_auc_train)
#     ks_train.append(total_ks_train)
#     num_samples.append(393294)
#     train_good.append(lgbm_mother_train_good)
#     train_bad.append(lgbm_mother_train_bad)
#     test_good.append(lgbm_mother_test_good)
#     test_bad.append(lgbm_mother_test_bad)
#     total_good.append(lgbm_mother_total_good)
#     total_bad.append(lgbm_mother_total_bad)
#     GB_thres.append(total_GBthres)
    
    model_result_df = pd.DataFrame({'月份组合':i_s,'test_good':test_good,'test_bad':test_bad,'test_auc':auc,'test_ks':ks,
                              'train_good':train_good,'train_bad':train_bad,'train_auc':auc_train,'train_ks':ks_train,
                              'total_good':total_good,'total_bad':total_bad,'GB_thres':GB_thres}).sort_values(by=['test_ks','test_auc'],ascending=False)
    # 'num_sample':num_samples,
    model_result_df['train_test_gap'] = model_result_df['train_auc'] - model_result_df['test_auc']
    feature_importance_df = pd.DataFrame(ftimps)
    
    return model_result_df,feature_importance_df,lgbm_preds_dfs,lgbm_clfs

In [109]:
lgbm_monthly_result,lgbm3_ftimp,lgbm_preds_df,lgbm_clfs5_6 = MakeLGBMResult(lgbm_recurrent_month)

In [110]:
lgbm_monthly_result['GB_thres'] = 1-lgbm_monthly_result['GB_thres']

In [111]:
lgbm_monthly_result

,月份组合,test_good,test_bad,test_auc,test_ks,train_good,train_bad,train_auc,train_ks,total_good,total_bad,GB_thres,train_test_gap
0,pair5,195157,17951,0.820796,0.488581,428193,50769,0.824326,0.490728,623350,68720,0.896617,0.003530
1,pair6,242944,29404,0.782193,0.419713,434781,47052,0.847045,0.528897,677725,76456,0.916463,0.064853


In [112]:
lgbm_monthly_result.to_csv('Monthly_rolling/pair5and6_result.csv',index=False)

In [114]:
lgbm_preds_df['pair6'].to_csv('Monthly_rolling/pair6_predN.csv',index=False)

In [119]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X_GB.drop('month_mark',axis=1))
pair1_predGB = lgbm_clfs5_6['pair6'].predict_proba(X)[:,1]

In [120]:
pd.concat([big_sheet_GB[['id','month_mark']],
           pd.DataFrame(pair1_predGB)],axis=1).to_csv('Monthly_rolling/pair6_predGB.csv',index=False)

In [ ]:
lgbm_preds_combined = pd.concat([lgbm_preds_df[4],lgbm_preds_df[5],lgbm_preds_df[7],lgbm_preds_df[8],
                                 lgbm_preds_df[11],lgbm_preds_df[12],lgbm_preds_df[14],lgbm_preds_df[15],
                                 lgbm_preds_df[19],lgbm_preds_df[20],lgbm_preds_df[22],lgbm_preds_df[23],
                                 lgbm_preds_df[26],lgbm_preds_df[27],lgbm_preds_df[29],lgbm_preds_df[30]],axis=0)

In [ ]:
lgbm_preds_combined = pd.concat([lgbm_preds_df[4],lgbm_preds_df[5],lgbm_preds_df[7],lgbm_preds_df[8],
                                 lgbm_preds_df[11],lgbm_preds_df[12],lgbm_preds_df[15],lgbm_preds_df[16],
                                 lgbm_preds_df[18],lgbm_preds_df[19],lgbm_preds_df[24],lgbm_preds_df[25],
                                 lgbm_preds_df[27],lgbm_preds_df[28],lgbm_preds_df[31],lgbm_preds_df[32],
                                 lgbm_preds_df[34],lgbm_preds_df[35],lgbm_preds_df[38],lgbm_preds_df[40],
                                 lgbm_preds_df[41],lgbm_preds_df[44],lgbm_preds_df[45],lgbm_preds_df[47],
                                 lgbm_preds_df[48]],axis=0)

In [ ]:
lgbm_preds_combined = pd.concat([lgbm_preds_df[4],lgbm_preds_df[5],lgbm_preds_df[8],lgbm_preds_df[9],
                                 lgbm_preds_df[11],lgbm_preds_df[12],lgbm_preds_df[17],lgbm_preds_df[19],
                                 lgbm_preds_df[20],lgbm_preds_df[23],lgbm_preds_df[24],lgbm_preds_df[26],
                                 lgbm_preds_df[27],lgbm_preds_df[32],lgbm_preds_df[33],lgbm_preds_df[35],
                                 lgbm_preds_df[36],lgbm_preds_df[39],lgbm_preds_df[40],lgbm_preds_df[42],
                                 lgbm_preds_df[43],lgbm_preds_df[46],lgbm_preds_df[48],lgbm_preds_df[49],
                                 lgbm_preds_df[52],lgbm_preds_df[53],lgbm_preds_df[55],lgbm_preds_df[56]],axis=0)

In [ ]:
sum(lgbm_preds_combined['pred_good_prob']>=0.5)

In [ ]:
lgbm_preds_combined.to_csv('lgbm25_NoR_pred_0427.csv',index=False)

In [ ]:
lgbm_preds_combined.to_csv('lgbm16_addR_pred_0427.csv',index=False)

In [ ]:
lgbm_preds_combined.to_csv('lgbm29_addR_pred_0427.csv',index=False)

In [ ]:
lgbm_preds_combined.shape

In [ ]:
sum(lgbm_preds_combined['pred_good_prob']<0.5)

**XGB**

In [ ]:
mother_X = big_sheet_GB.drop(['id','nasrdw_recd_date','bad'],axis=1)
mother_Y = big_sheet_GB['bad'].replace({'G':1,'B':0})
mother_N = big_sheet_N
[xgb_mother_clf,xgb_mother_auc,xgb_mother_ks,xgb_mother_y_test,xgb_mother_pred,
 xgb_mother_auc_train,xgb_mother_ks_train,xgb_mother_y_train,xgb_mother_pred_train,
 xgb_mother_train_good,xgb_mother_train_bad,xgb_mother_test_good,xgb_mother_test_bad,
 xgb_mother_total_good,xgb_mother_total_bad,mother_ftimps,xgb_mother_NRpred] = XGB(mother_X,mother_Y,mother_N,'mother',xgb_clf0)
mother_Y.value_counts(),xgb_mother_auc,xgb_mother_ks,xgb_mother_auc_train,xgb_mother_ks_train

In [ ]:
end_nodes = outside_data_tree3.end_nodes
nodes_id = end_nodes.keys()

list1 = [9,8]
list2 = [11,12,5,4,2]

xgb3_return_dict={}
for i in nodes_id:
    
    X_GB = big_sheet_GB.iloc[end_nodes[i],:].drop(['id','bad'],axis=1)
    Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0})  
    
    if i in list1:
        xgb3_return_dict[i] = XGB(X_GB,Y_GB,i,xgb_clf1)
    else:
        xgb3_return_dict[i] = XGB(X_GB,Y_GB,i,xgb_clf2)

In [ ]:
xgb3_result,xgb3_ftimp = MakeLGBMResult(xgb3_return_dict,outside_data_tree3)
xgb3_result['train_test_gap'] = xgb3_result['train_auc']-xgb3_result['test_auc']
xgb3_result.sort_values(by='train_test_gap')

**IV+LR**

In [ ]:
mother_X = big_sheet_GB[mother_iv_fts]
mother_Y = big_sheet_GB['bad'].replace({'G':1,'B':0})
[mother_clf,mother_auc,mother_ks,mother_y_test,mother_pred,
 mother_auc_train,mother_ks_train,mother_y_train,mother_pred_train,
 mother_used_fts,
 mother_train_good,mother_train_bad,mother_test_good,mother_test_bad,mother_total_good,mother_total_bad,
 mother_coefs] = LR(mother_X,mother_Y,'mother',{})
mother_Y.value_counts(),mother_auc,mother_ks,mother_auc_train,mother_ks_train

In [ ]:
start = time.time()
end_nodes_iv3 = outside_data_tree3.end_nodes
end_nodes_iv_keys = list(end_nodes_iv3.keys())

if __name__ == '__main__':
    manager = mp.Manager()
    iv_dict3 = manager.dict()
    jobs = []
    for i in end_nodes_iv_keys:
        
        X_GB = big_sheet_GB.iloc[end_nodes_iv3[i],:].drop(['id','bad'],axis=1)
        Y_GB = big_sheet_GB['bad'].iloc[end_nodes_iv3[i]].replace({'G':1,'B':0})
        IV_cols = list(X_GB.columns)
        p = mp.Process(target=Run_IV, args=(X_GB,Y_GB,IV_cols,iv_dict3,i))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
        
end = time.time()
print('whole IV procedure takes {}seconds'.format(np.round(end-start,2)))

In [ ]:
start = time.time()
end_nodes_iv4 = outside_data_tree4.end_nodes
end_nodes_iv_keys = list(end_nodes_iv4.keys())

if __name__ == '__main__':
    manager = mp.Manager()
    iv_dict4 = manager.dict()
    jobs = []
    for i in end_nodes_iv_keys:
        
        X_GB = big_sheet_GB.iloc[end_nodes_iv4[i],:].drop(['id','bad'],axis=1)
        Y_GB = big_sheet_GB['bad'].iloc[end_nodes_iv4[i]].replace({'G':1,'B':0})
        IV_cols = list(X_GB.columns)
        p = mp.Process(target=Run_IV, args=(X_GB,Y_GB,IV_cols,iv_dict4,i))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
        
end = time.time()
print('whole IV procedure takes {}seconds'.format(np.round(end-start,2)))

In [ ]:
start = time.time()
end_nodes_iv5 = outside_data_tree5.end_nodes
end_nodes_iv_keys = list(end_nodes_iv5.keys())

if __name__ == '__main__':
    manager = mp.Manager()
    iv_dict5 = manager.dict()
    jobs = []
    for i in end_nodes_iv_keys:
        
        X_GB = big_sheet_GB.iloc[end_nodes_iv5[i],:].drop(['id','bad'],axis=1)
        Y_GB = big_sheet_GB['bad'].iloc[end_nodes_iv5[i]].replace({'G':1,'B':0})
        IV_cols = list(X_GB.columns)
        p = mp.Process(target=Run_IV, args=(X_GB,Y_GB,IV_cols,iv_dict5,i))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
        
end = time.time()
print('whole IV procedure takes {}seconds'.format(np.round(end-start,2)))

In [ ]:
iv_thres=0

In [ ]:
children_iv_fts3 = {}
for i in list(iv_dict3.keys()):
    cond = iv_dict3[i]>iv_thres
    children_iv_fts3[i] = list(iv_dict3[i].loc[cond].index.values)

In [ ]:
children_iv_fts4 = {}
for i in list(iv_dict4.keys()):
    cond = iv_dict4[i]>iv_thres
    children_iv_fts4[i] = list(iv_dict4[i].loc[cond].index.values)

In [ ]:
children_iv_fts5 = {}
for i in list(iv_dict5.keys()):
    cond = iv_dict5[i]>iv_thres
    children_iv_fts5[i] = list(iv_dict5[i].loc[cond].index.values)

In [ ]:
start = time.time()
end_nodes = outside_data_tree3.end_nodes
nodes_id = end_nodes.keys()
if __name__ == '__main__':
    manager = mp.Manager()
    lr3_return_dict = manager.dict()
    jobs = []
    for i in nodes_id:
        
        X_GB = big_sheet_GB[children_iv_fts3[i]].iloc[end_nodes[i],:]
        Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0})
        
        p = mp.Process(target=LR, args=(X_GB,Y_GB,i,lr3_return_dict))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
        
end = time.time()
print('total time cost of duplicates processing takes {}seconds'.format(np.round(end-start,2)))

In [ ]:
start = time.time()
end_nodes = outside_data_tree4.end_nodes
nodes_id = end_nodes.keys()
if __name__ == '__main__':
    manager = mp.Manager()
    lr4_return_dict = manager.dict()
    jobs = []
    for i in nodes_id:
        
        X_GB = big_sheet_GB[children_iv_fts4[i]].iloc[end_nodes[i],:]
        Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0})
        
        p = mp.Process(target=LR, args=(X_GB,Y_GB,i,lr4_return_dict))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
        
end = time.time()
print('total time cost of duplicates processing takes {}seconds'.format(np.round(end-start,2)))

In [ ]:
start = time.time()
end_nodes = outside_data_tree5.end_nodes
nodes_id = end_nodes.keys()
if __name__ == '__main__':
    manager = mp.Manager()
    lr5_return_dict = manager.dict()
    jobs = []
    for i in nodes_id:
        
        X_GB = big_sheet_GB[children_iv_fts5[i]].iloc[end_nodes[i],:]
        Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0})
        
        p = mp.Process(target=LR, args=(X_GB,Y_GB,i,lr5_return_dict))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
        
end = time.time()
print('total time cost of duplicates processing takes {}seconds'.format(np.round(end-start,2)))

In [ ]:
auc,auc_train,i_s,ks,ks_train,num_samples = [],[],[],[],[],[]

train_good,train_bad,test_good,test_bad,total_good,total_bad = [],[],[],[],[],[]

used_fts = {}
coefs = {}
coefs['fts'] = mother_X.columns
coefs['mother']=mother_coefs

for i in lr3_return_dict.keys():
    i_s.append(i)
    auc.append(lr3_return_dict[i][1])
#     cv_score.append(sum(model_result[i]['cv_score'])/len(model_result[i]['cv_score']))
    ks.append(lr3_return_dict[i][2])
    auc_train.append(lr3_return_dict[i][5])
    ks_train.append(lr3_return_dict[i][6])
    num_samples.append(len(outside_data_tree3.end_nodes[i]))
    train_good.append(lr3_return_dict[i][10])
    train_bad.append(lr3_return_dict[i][11])
    test_good.append(lr3_return_dict[i][12])
    test_bad.append(lr3_return_dict[i][13])
    total_good.append(lr3_return_dict[i][14])
    total_bad.append(lr3_return_dict[i][15])
    used_fts[i] = lr3_return_dict[i][9]
    coefs[i]=lr3_return_dict[i][16]
    
i_s.append('mother')
auc.append(mother_auc)
ks.append(mother_ks)
auc_train.append(mother_auc_train)
ks_train.append(mother_ks_train)
num_samples.append(393294)
train_good.append(mother_train_good)
train_bad.append(mother_train_bad)
test_good.append(mother_test_good)
test_bad.append(mother_test_bad)
total_good.append(mother_total_good)
total_bad.append(mother_total_bad)

y_test_stack = np.array([])
y_pred_stack = np.array([])
y_pred_train_stack = np.array([])
y_train_stack = np.array([])
for i in lr3_return_dict.keys():
    y_test_stack = np.concatenate([y_test_stack,lr3_return_dict[i][3]])
    y_pred_stack = np.concatenate([y_pred_stack,lr3_return_dict[i][4]])
    y_pred_train_stack = np.concatenate([y_pred_train_stack,lr3_return_dict[i][8]])
    y_train_stack = np.concatenate([y_train_stack,lr3_return_dict[i][7]])
    
total_auc = roc_auc_score(y_test_stack,y_pred_stack)
fpr,tpr,thresholds= roc_curve(y_test_stack,y_pred_stack)
total_ks = max(tpr-fpr)

total_auc_train = roc_auc_score(y_train_stack,y_pred_train_stack)
fpr,tpr,thresholds= roc_curve(y_train_stack,y_pred_train_stack)
total_ks_train = max(tpr-fpr)

i_s.append('子群整合')
auc.append(total_auc)
ks.append(total_ks)
auc_train.append(total_auc_train)
ks_train.append(total_ks_train)
num_samples.append(393294)
train_good.append(mother_train_good)
train_bad.append(mother_train_bad)
test_good.append(mother_test_good)
test_bad.append(mother_test_bad)
total_good.append(mother_total_good)
total_bad.append(mother_total_bad)

total_auc,total_ks,total_auc_train,total_ks_train
lr3_480_0423_iv_result_df = pd.DataFrame({'子群ID':i_s,'test_good':test_good,'test_bad':test_bad,'test_auc':auc,'test_ks':ks,
                              'train_good':train_good,'train_bad':train_bad,'auc_train':auc_train,'ks_train':ks_train,
                              'total_good':total_good,'total_bad':total_bad,'num_sample':num_samples}).sort_values(by=['test_ks','test_auc'],ascending=False)

lr3_480_0423_iv_result_df.to_csv('lr3_iv_result_df_480_0423.csv')

keys = list(children_iv_fts3.keys())

lr3_480_iv_weights1 = pd.DataFrame([mother_iv_fts,mother_coefs]).T
lr3_480_iv_weights1.columns = ['mother_fts','mother']

lr3_480_iv_weights2 = pd.DataFrame([mother_iv_fts,mother_coefs]).T
lr3_480_iv_weights2.columns = ['mother_fts','mother']

for i in keys:
    df = pd.DataFrame([children_iv_fts3[i],coefs[i]]).T
    df.columns = ['mother_fts',str(i)+'_weights']
    lr3_480_iv_weights1 = pd.merge(left =lr3_480_iv_weights1,right = df,
                                   on='mother_fts',
                                   how='outer',copy=False)
    
    df.columns = [str(i),str(i)+'_weights']
    lr3_480_iv_weights2 = pd.concat([lr3_480_iv_weights2,df],axis=1)

lr3_480_iv_weights1.fillna(0,inplace=True)
lr3_480_iv_weights1.to_csv('lr3_iv_480_weights_verion1.csv',index=False,encoding='GBK')
lr3_480_iv_weights2.to_csv('lr3_iv_480_weights_verion2.csv',index=False,encoding='GBK')

**IV+LGBM**

In [ ]:
# xgb_clf,auc,ks,y_test,pred[:,1],auc_train,ks_train,y_train,
# pred_train[:,1],used_fts,train_good,train_bad,test_good,test_bad,total_good,total_bad
mother_X = big_sheet_GB[mother_iv_fts]
mother_Y = big_sheet_GB['bad'].replace({'G':1,'B':0})
[lgbm_mother_clf,lgbm_mother_auc,lgbm_mother_ks,lgbm_mother_y_test,lgbm_mother_pred,
 lgbm_mother_auc_train,lgbm_mother_ks_train,lgbm_mother_y_train,lgbm_mother_pred_train,
 lgbm_mother_train_good,lgbm_mother_train_bad,lgbm_mother_test_good,lgbm_mother_test_bad,
 lgbm_mother_total_good,lgbm_mother_total_bad,mother_lgbm_ftimp] = LGBM(mother_X,mother_Y,'mother')
mother_Y.value_counts(),lgbm_mother_auc,lgbm_mother_ks,lgbm_mother_auc_train,lgbm_mother_ks_train

In [ ]:
end_nodes = outside_data_tree3.end_nodes
nodes_id = end_nodes.keys()

lgbm3_return_dict={}
for i in nodes_id:
    
    X_GB = big_sheet_GB[children_iv_fts3[i]].iloc[end_nodes[i],:]
    Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0}) 
    lgbm3_return_dict[i] = LGBM(X_GB,Y_GB,i)

In [ ]:
end_nodes = outside_data_tree4.end_nodes
nodes_id = end_nodes.keys()

lgbm4_return_dict={}
for i in nodes_id:
    
    X_GB = big_sheet_GB[children_iv_fts4[i]].iloc[end_nodes[i],:]
    Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0}) 
    lgbm4_return_dict[i] = LGBM(X_GB,Y_GB,i)

In [ ]:
end_nodes = outside_data_tree5.end_nodes
nodes_id = end_nodes.keys()

lgbm5_return_dict={}
for i in nodes_id:
    
    X_GB = big_sheet_GB[children_iv_fts5[i]].iloc[end_nodes[i],:]
    Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0}) 
    lgbm5_return_dict[i] = LGBM(X_GB,Y_GB,i)

In [ ]:
auc,auc_train,i_s,ks,ks_train,num_samples = [],[],[],[],[],[]

train_good,train_bad,test_good,test_bad,total_good,total_bad = [],[],[],[],[],[]

ftimp = {}
ftimp['mother'] = mother_lgbm_ftimp

for i in lgbm5_return_dict.keys():
    i_s.append(i)
    auc.append(lgbm5_return_dict[i][1])
#     cv_score.append(sum(model_result[i]['cv_score'])/len(model_result[i]['cv_score']))
    ks.append(lgbm5_return_dict[i][2])
    auc_train.append(lgbm5_return_dict[i][5])
    ks_train.append(lgbm5_return_dict[i][6])
    num_samples.append(len(outside_data_tree5.end_nodes[i]))
    train_good.append(lgbm5_return_dict[i][9])
    train_bad.append(lgbm5_return_dict[i][10])
    test_good.append(lgbm5_return_dict[i][11])
    test_bad.append(lgbm5_return_dict[i][12])
    total_good.append(lgbm5_return_dict[i][13])
    total_bad.append(lgbm5_return_dict[i][14])
    ftimp[i] = lgbm5_return_dict[i][-1]
    
i_s.append('mother')
auc.append(lgbm_mother_auc)
ks.append(lgbm_mother_ks)
auc_train.append(lgbm_mother_auc_train)
ks_train.append(lgbm_mother_ks_train)
num_samples.append(393294)
train_good.append(lgbm_mother_train_good)
train_bad.append(lgbm_mother_train_bad)
test_good.append(lgbm_mother_test_good)
test_bad.append(lgbm_mother_test_bad)
total_good.append(lgbm_mother_total_good)
total_bad.append(lgbm_mother_total_bad)

y_test_stack = np.array([])
y_pred_stack = np.array([])
y_pred_train_stack = np.array([])
y_train_stack = np.array([])
for i in lgbm5_return_dict.keys():
    y_test_stack = np.concatenate([y_test_stack,lgbm5_return_dict[i][3]])
    y_pred_stack = np.concatenate([y_pred_stack,lgbm5_return_dict[i][4]])
    y_pred_train_stack = np.concatenate([y_pred_train_stack,lgbm5_return_dict[i][8]])
    y_train_stack = np.concatenate([y_train_stack,lgbm5_return_dict[i][7]])
    
total_auc = roc_auc_score(y_test_stack,y_pred_stack)
fpr,tpr,thresholds= roc_curve(y_test_stack,y_pred_stack)
total_ks = max(tpr-fpr)

total_auc_train = roc_auc_score(y_train_stack,y_pred_train_stack)
fpr,tpr,thresholds= roc_curve(y_train_stack,y_pred_train_stack)
total_ks_train = max(tpr-fpr)

i_s.append('子群整合')
auc.append(total_auc)
ks.append(total_ks)
auc_train.append(total_auc_train)
ks_train.append(total_ks_train)
num_samples.append(393294)
train_good.append(lgbm_mother_train_good)
train_bad.append(lgbm_mother_train_bad)
test_good.append(lgbm_mother_test_good)
test_bad.append(lgbm_mother_test_bad)
total_good.append(lgbm_mother_total_good)
total_bad.append(lgbm_mother_total_bad)

total_auc,total_ks,total_auc_train,total_ks_train
lgbm5_iv_480_0423_result_df = pd.DataFrame({'子群ID':i_s,'test_good':test_good,'test_bad':test_bad,'test_auc':auc,'test_ks':ks,
                              'train_good':train_good,'train_bad':train_bad,'train_auc':auc_train,'train_ks':ks_train,
                              'total_good':total_good,'total_bad':total_bad,'num_sample':num_samples}).sort_values(by=['test_ks','test_auc'],ascending=False)

lgbm5_iv_480_0423_result_df.to_csv('lgbm5_iv_result_df_480_0423.csv',index=False)

keys = list(children_iv_fts5.keys())

lgbm5_480_iv_weights1 = pd.DataFrame([mother_iv_fts,mother_lgbm_ftimp]).T
lgbm5_480_iv_weights1.columns = ['mother_fts','mother_importances']

lgbm5_480_iv_weights2 = pd.DataFrame([mother_iv_fts,mother_lgbm_ftimp]).T
lgbm5_480_iv_weights2.columns = ['mother_fts','mother_importances']

for i in keys:
    df = pd.DataFrame([children_iv_fts5[i],ftimp[i]]).T
    df.columns = ['mother_fts',str(i)+'_importances']
    lgbm5_480_iv_weights1 = pd.merge(left =lgbm5_480_iv_weights1,right = df,
                                   on='mother_fts',
                                   how='outer',copy=False)
    
    df.columns = [str(i),str(i)+'_importances']
    lgbm5_480_iv_weights2 = pd.concat([lgbm5_480_iv_weights2,df],axis=1)
    
lgbm5_480_iv_weights1.fillna(0,inplace=True)
lgbm5_480_iv_weights1.to_csv('lgbm5_480_0423_iv_weights_version1.csv',index=False,encoding='GBK')
lgbm5_480_iv_weights2.to_csv('lgbm5_480_0423_iv_weights_version2.csv',index=False,encoding='GBK')

**IV+XGB**

In [ ]:
mother_X = big_sheet_GB[mother_iv_fts]
mother_Y = big_sheet_GB['bad'].replace({'G':1,'B':0})
[xgb_mother_clf,xgb_mother_auc,xgb_mother_ks,xgb_mother_y_test,xgb_mother_pred,
 xgb_mother_auc_train,xgb_mother_ks_train,xgb_mother_y_train,xgb_mother_pred_train,
 xgb_mother_train_good,xgb_mother_train_bad,xgb_mother_test_good,xgb_mother_test_bad,
 xgb_mother_total_good,xgb_mother_total_bad,mother_xgb_ftimps] = XGB(mother_X,mother_Y,'mother',params)
mother_Y.value_counts(),xgb_mother_auc,xgb_mother_ks,xgb_mother_auc_train,xgb_mother_ks_train

In [ ]:
end_nodes = outside_data_tree3.end_nodes
nodes_id = end_nodes.keys()

xgb3_return_dict={}
for i in nodes_id:
    
    X_GB = big_sheet_GB[children_iv_fts3[i]].iloc[end_nodes[i],:]
    Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0}) 
    xgb3_return_dict[i] = XGB(X_GB,Y_GB,i,params)

In [ ]:
end_nodes = outside_data_tree4.end_nodes
nodes_id = end_nodes.keys()

xgb4_return_dict={}
for i in nodes_id:
    
    X_GB = big_sheet_GB[children_iv_fts4[i]].iloc[end_nodes[i],:]
    Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0}) 
    xgb4_return_dict[i] = XGB(X_GB,Y_GB,i,params)

In [ ]:
end_nodes = outside_data_tree5.end_nodes
nodes_id = end_nodes.keys()

xgb5_return_dict={}
for i in nodes_id:
    
    X_GB = big_sheet_GB[children_iv_fts5[i]].iloc[end_nodes[i],:]
    Y_GB = big_sheet_GB['bad'].iloc[end_nodes[i]].replace({'G':1,'B':0}) 
    xgb5_return_dict[i] = XGB(X_GB,Y_GB,i,params)

In [ ]:
auc,auc_train,i_s,ks,ks_train,num_samples = [],[],[],[],[],[]

train_good,train_bad,test_good,test_bad,total_good,total_bad = [],[],[],[],[],[]

ftimp = {}
ftimp['mother'] = mother_xgb_ftimps

for i in xgb5_return_dict.keys():
    i_s.append(i)
    auc.append(xgb5_return_dict[i][1])
#     cv_score.append(sum(model_result[i]['cv_score'])/len(model_result[i]['cv_score']))
    ks.append(xgb5_return_dict[i][2])
    auc_train.append(xgb5_return_dict[i][5])
    ks_train.append(xgb5_return_dict[i][6])
    num_samples.append(len(outside_data_tree5.end_nodes[i]))
    train_good.append(xgb5_return_dict[i][9])
    train_bad.append(xgb5_return_dict[i][10])
    test_good.append(xgb5_return_dict[i][11])
    test_bad.append(xgb5_return_dict[i][12])
    total_good.append(xgb5_return_dict[i][13])
    total_bad.append(xgb5_return_dict[i][14])
    ftimp[i] = xgb5_return_dict[i][-1]
    
i_s.append('mother')
auc.append(xgb_mother_auc)
ks.append(xgb_mother_ks)
auc_train.append(xgb_mother_auc_train)
ks_train.append(xgb_mother_ks_train)
num_samples.append(393294)
train_good.append(xgb_mother_train_good)
train_bad.append(xgb_mother_train_bad)
test_good.append(xgb_mother_test_good)
test_bad.append(xgb_mother_test_bad)
total_good.append(xgb_mother_total_good)
total_bad.append(xgb_mother_total_bad)

y_test_stack = np.array([])
y_pred_stack = np.array([])
y_pred_train_stack = np.array([])
y_train_stack = np.array([])
for i in xgb5_return_dict.keys():
    y_test_stack = np.concatenate([y_test_stack,xgb5_return_dict[i][3]])
    y_pred_stack = np.concatenate([y_pred_stack,xgb5_return_dict[i][4]])
    y_pred_train_stack = np.concatenate([y_pred_train_stack,xgb5_return_dict[i][8]])
    y_train_stack = np.concatenate([y_train_stack,xgb5_return_dict[i][7]])
    
total_auc = roc_auc_score(y_test_stack,y_pred_stack)
fpr,tpr,thresholds= roc_curve(y_test_stack,y_pred_stack)
total_ks = max(tpr-fpr)

total_auc_train = roc_auc_score(y_train_stack,y_pred_train_stack)
fpr,tpr,thresholds= roc_curve(y_train_stack,y_pred_train_stack)
total_ks_train = max(tpr-fpr)

i_s.append('子群整合')
auc.append(total_auc)
ks.append(total_ks)
auc_train.append(total_auc_train)
ks_train.append(total_ks_train)
num_samples.append(393294)
train_good.append(xgb_mother_train_good)
train_bad.append(xgb_mother_train_bad)
test_good.append(xgb_mother_test_good)
test_bad.append(xgb_mother_test_bad)
total_good.append(xgb_mother_total_good)
total_bad.append(xgb_mother_total_bad)

total_auc,total_ks,total_auc_train,total_ks_train
xgb5_iv_480_0423_result_df = pd.DataFrame({'子群ID':i_s,'test_good':test_good,'test_bad':test_bad,'test_auc':auc,'test_ks':ks,
                              'train_good':train_good,'train_bad':train_bad,'train_auc':auc_train,'train_ks':ks_train,
                              'total_good':total_good,'total_bad':total_bad,'num_sample':num_samples}).sort_values(by=['test_ks','test_auc'],ascending=False)

xgb5_iv_480_0423_result_df.to_csv('xgb5_iv_result_df_480_0423.csv',index=False)

keys = list(children_iv_fts5.keys())

xgb5_480_iv_weights1 = pd.DataFrame([mother_iv_fts,mother_xgb_ftimps]).T
xgb5_480_iv_weights1.columns = ['mother_fts','mother_importances']

xgb5_480_iv_weights2 = pd.DataFrame([mother_iv_fts,mother_xgb_ftimps]).T
xgb5_480_iv_weights2.columns = ['mother_fts','mother_importances']

for i in keys:
    df = pd.DataFrame([children_iv_fts5[i],ftimp[i]]).T
    df.columns = ['mother_fts',str(i)+'_importances']
    xgb5_480_iv_weights1 = pd.merge(left =xgb5_480_iv_weights1,right = df,
                                   on='mother_fts',
                                   how='outer',copy=False)
    
    df.columns = [str(i),str(i)+'_importances']
    xgb5_480_iv_weights2 = pd.concat([xgb5_480_iv_weights2,df],axis=1)
    
xgb5_480_iv_weights1.fillna(0,inplace=True)
xgb5_480_iv_weights1.to_csv('xgb5_480_0423_iv_weights_version1.csv',index=False,encoding='GBK')
xgb5_480_iv_weights2.to_csv('xgb5_480_0423_iv_weights_version2.csv',index=False,encoding='GBK')

In [ ]:
xgb3_iv_480_0423_result_df

**预测**

In [ ]:
childrens_NR3 = outside_data_tree3.NR_nodes
childrens_NR4 = outside_data_tree4.NR_nodes
childrens_NR5 = outside_data_tree5.NR_nodes

In [ ]:
split8_NR_IDs = {}

for i in childrens_NR3.keys():
    split8_NR_IDs[i] = outside_data_tree3.df_NR.iloc[childrens_NR3[i]][['id','bad']]
    
split8_NR_X = {}

for i in split8_NR_IDs.keys():
    
    df = pd.merge(left = big_sheet_NR.drop('bad',axis=1),
                  right = split8_NR_IDs[i].drop('bad',axis=1),
                  on='id',
                  how='inner')
    
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df.drop('id',axis=1))
    split8_NR_X[i] = {}
    split8_NR_X[i]['x'] = scaled
    split8_NR_X[i]['df'] = df
                                  
    
pred8_NR = {}

for i in split8_NR_X.keys():
    pred8_NR[i] = {}
    pred8_NR[i]['id'] = split8_NR_X[i]['df']['id']
    pred8_NR[i]['pred_prob']= lgbm3_return_dict[i][0].predict_proba(split8_NR_X[i]['x'])[:,1]
    
pred8_NR_df = {}

for i in pred8_NR.keys():
    
    temp_df = pd.DataFrame(pred8_NR[i])
    temp_df['split id'] = i
    pred8_NR_df[i] = temp_df

In [ ]:
lgbm3_return_dict[4][0].predict_proba(split8_NR_X[4]['x'])[:,1]

In [ ]:
lgbm3_return_dict[3][0].predict_proba(split8_NR_X[4]['x'])[:,1]

In [ ]:
pred8_NR_df.keys()

In [ ]:
pred_df = pd.concat([pred8_NR_df[3],pred8_NR_df[4],pred8_NR_df[6],pred8_NR_df[7],
                     pred8_NR_df[10],pred8_NR_df[11],pred8_NR_df[13],pred8_NR_df[14]],axis=0)

In [ ]:
sum(pred_df['pred_prob']<0.55)

In [ ]:
pred_df.to_csv('lr8_pred_0426.csv',index=False)

In [ ]:
sum(pred_df['pred_prob']<0.55)

**Monthly Recurrent Estimation**

In [150]:
pair1_pred_N = pd.read_csv('Monthly_rolling/pair1_predN.csv').iloc[:,:-1]
pair1_pred_N = mm.reduce_mem_usage(pair1_pred_N)
pair1_pred_N.columns = ['id','pair1_predN']

pair2_pred_N = pd.read_csv('Monthly_rolling/pair2_predN.csv').iloc[:,:-1]
pair2_pred_N = mm.reduce_mem_usage(pair2_pred_N)
pair2_pred_N.columns = ['id','pair2_predN']

pair3_pred_N = pd.read_csv('Monthly_rolling/pair3_predN.csv').iloc[:,:-1]
pair3_pred_N = mm.reduce_mem_usage(pair3_pred_N)
pair3_pred_N.columns = ['id','pair3_predN']

pair4_pred_N = pd.read_csv('Monthly_rolling/pair4_predN.csv').iloc[:,:-1]
pair4_pred_N = mm.reduce_mem_usage(pair4_pred_N)
pair4_pred_N.columns = ['id','pair4_predN']

pair5_pred_N = pd.read_csv('Monthly_rolling/pair5_predN.csv').iloc[:,:-1]
pair5_pred_N = mm.reduce_mem_usage(pair5_pred_N)
pair5_pred_N.columns = ['id','pair5_predN']

pair6_pred_N = pd.read_csv('Monthly_rolling/pair6_predN.csv').iloc[:,:-1]
pair6_pred_N = mm.reduce_mem_usage(pair6_pred_N)
pair6_pred_N.columns = ['id','pair6_predN']

Reduce Memory Usage Function Reports:
Memory usage of dataframe is 6.88 MB
Memory usage after optimization is: 4.30 MB
Decreased by 37.5%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 6.88 MB
Memory usage after optimization is: 4.30 MB
Decreased by 37.5%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 6.88 MB
Memory usage after optimization is: 4.30 MB
Decreased by 37.5%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 6.88 MB
Memory usage after optimization is: 4.30 MB
Decreased by 37.5%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 6.88 MB
Memory usage after optimization is: 4.30 MB
Decreased by 37.5%
 
Reduce Memory Usage Function Reports:
Memory usage of dataframe is 6.88 MB
Memory usage after optimization is: 4.30 MB
Decreased by 37.5%
 


In [151]:
monthly_pred_N = pd.merge(pair1_pred_N,pair2_pred_N,on='id',how='inner')
monthly_pred_N = pd.merge(monthly_pred_N,pair3_pred_N,on='id',how='inner')
monthly_pred_N = pd.merge(monthly_pred_N,pair4_pred_N,on='id',how='inner')
monthly_pred_N = pd.merge(monthly_pred_N,pair5_pred_N,on='id',how='inner')
monthly_pred_N = pd.merge(monthly_pred_N,pair6_pred_N,on='id',how='inner')

In [152]:
monthly_pred_N = mm.reduce_mem_usage(monthly_pred_N)

Reduce Memory Usage Function Reports:
Memory usage of dataframe is 12.03 MB
Memory usage after optimization is: 12.03 MB
Decreased by 0.0%
 


In [154]:
monthly_pred_N['avg_predN'] = monthly_pred_N.iloc[:,1:7].sum(axis=1)/6

In [170]:
monthly_pred_GB = pd.read_csv('Monthly_rolling/monthly_rolling_predGB_combined.csv').drop('Unnamed: 0',axis=1)
monthly_pred_GB = mm.reduce_mem_usage(monthly_pred_GB)

Reduce Memory Usage Function Reports:
Memory usage of dataframe is 75.16 MB
Memory usage after optimization is: 30.07 MB
Decreased by 60.0%
 


In [172]:
monthly_pred_GB['4pairs_avg'] = (monthly_pred_GB['pair2_pred_good']+
                                 monthly_pred_GB['pair3_pred_good']+
                                 monthly_pred_GB['pair4_pred_good']+
                                 monthly_pred_GB['pair5_pred_good'])/4

In [173]:
monthly_pred_GB

,id,month_mark,pair1_pred_good,pair2_pred_good,pair3_pred_good,pair4_pred_good,pair5_pred_good,pair6_pred_good,avg_pred,bad,4pairs_avg
0,56a13c06ee2f2a0db827d0d9450213d3,2017,0.991211,0.984863,0.991211,0.993164,0.991211,0.993164,0.990723,G,0.990234
1,c6342448d324d0e99e0d89175a2da335,2017,0.997070,0.997559,0.997070,0.997559,0.997559,0.997070,0.997559,G,0.997559
2,82e5383d3b794f2555dadb7e37e9108e,2,0.982910,0.986328,0.988281,0.980469,0.979492,0.981445,0.982910,G,0.983398
3,a5e91382fc770d3bf1c3845d113d85a5,3,0.985840,0.988281,0.985352,0.986816,0.981934,0.978516,0.984375,G,0.985840
4,57e97bcbc4090d25d1c783fcb071b6df,3,0.993164,0.995605,0.995117,0.995117,0.994629,0.993164,0.994629,G,0.994629
...,...,...,...,...,...,...,...,...,...,...,...
985196,edb46b1f6087ba0f55fe280f217c9de0,12,0.945801,0.958496,0.957520,0.967773,0.975098,0.978027,0.963379,G,0.964355
985197,c17c9093f1bce691c6cc292d5af5eeb7,12,0.776855,0.801758,0.833984,0.835938,0.796875,0.772461,0.802734,G,0.817383
985198,fe203941c63fd6ca564c62a26ca6f6db,12,0.968262,0.966797,0.961914,0.967285,0.958984,0.956543,0.963379,G,0.963867
985199,8939c235823b02db54ade065d8f0e128,12,0.777832,0.792969,0.769043,0.843750,0.783691,0.781250,0.791504,G,0.797363


In [174]:
auc_ks_monthly = {}
for i in range(1,13):
    
    cond = monthly_pred_GB['month_mark'] == i
    pred = monthly_pred_GB.loc[cond]['4pairs_avg']
    Y_test = monthly_pred_GB['bad'].replace({'G':1,'B':0}).loc[cond]
    
    auc = roc_auc_score(Y_test,pred)
    fpr,tpr,thresholds= roc_curve(Y_test,pred)
    ks = max(tpr-fpr)
    GB_thres = thresholds[np.argmax(tpr-fpr)]
    auc_ks_monthly[str(i)] ={'auc':auc,'ks':ks,'GB_thres':GB_thres}

In [176]:
pd.DataFrame(auc_ks_monthly).T.to_csv('Monthly_rolling/monthly_performance_4pairs.csv')